In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.style import use as usestyle
from metpy.plots import SkewT
from metpy.units import units
import metpy.calc as mpcalc
from copy import deepcopy
from glob import glob
from matplotlib import colors, font_manager, _tight_layout, gridspec, rcParams
from matplotlib.colors import BoundaryNorm
from matplotlib.transforms import Bbox, TransformedBbox

In [ ]:
seriflist = glob(f"../liberation-serif/*.ttf")
for fontpath in seriflist:
    font_manager.fontManager.addfont(fontpath)
usestyle("paperplots.mplstyle")
rcParams["lines.linewidth"] = 1

In [ ]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        # Note also that we must extrapolate beyond vmin/vmax
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1.]
        return np.ma.masked_array(np.interp(value, x, y,
                                            left=-np.inf, right=np.inf))

    def inverse(self, value):
        y, x = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y, left=-np.inf, right=np.inf)

In [ ]:
def get_dwpt(w, p):
    vappres = w*p / (0.622 + w)
    tdew = (4717.20 - 35.86 * np.log(vappres/6.1078)) / (17.2693882 - np.log(vappres/6.1078))
    return tdew
#Return dewpoint temperature in K based on water vapor mixing ratio in kg/kg and pressure in hPa

In [ ]:
def add_subplotlabel(ax, axlabel, left, width, bottom, height, fs = 8):
    right = left + width
    top = bottom + height
    p = plt.Rectangle((left, bottom), width, height, fill=True, zorder = 3, edgecolor = "black", linewidth = 0.2, facecolor = "white")
    p.set_transform(ax.transAxes)
    p.set_clip_on(False)
    ax.add_patch(p)
    ax.text(left+0.5*width, bottom+0.5*height, axlabel, fontsize = fs, transform = ax.transAxes, horizontalalignment = "center", verticalalignment = "center")
    return ax

In [ ]:
def isdac_snd_control():
    dz = 5
    zarray = np.arange(0, 2200., dz)
    tarray = np.zeros_like(zarray)
    qvarray = np.zeros_like(zarray)
    qlarray = np.zeros_like(zarray)
    parray = np.zeros_like(zarray)
    theta0 = 263.4
    qvarray[0] = 1.8
    qlarray[0] = 0
    parray[0] = 102000
    tarray[0] = theta0 * (parray[0]/100000)**(0.286)
    for i,z in enumerate(zarray[1:]):
        rhobelow = parray[i] / (287 * tarray[i]* (1 +  0.61 * 1/1000 * qvarray[i]))
        dpdz = -rhobelow * 9.81
        pressure = parray[i] + dpdz * dz
        parray[i+1] = pressure
        # print(pressure)
        if z < 400:
            qt = 1.5 - 0.00075 * ( z - 400.0 )
            theta = 265 + 0.004 * (z - 400)
            tarray[i+1] = theta * (pressure / 100000)**(0.286)
            qvarray[i+1] = min(qt, 1000*qsatw(tarray[i+1], pressure/100))
            qlarray[i+1] = max(qt - qvarray[i+1], 0)
        elif z < 825:
            qt = 1.5
            theta = 265
            tarray[i+1] = theta * (pressure / 100000)**(0.286)
            qvarray[i+1] = min(qt, 1000*qsatw(tarray[i+1], pressure/100))
            qlarray[i+1] = max(qt - qvarray[i+1], 0)
        elif z < 2045:
            qt = 1.2
            theta = 266 + (z - 825)**(0.3)
            tarray[i+1] = theta * (pressure / 100000)**(0.286)
            qvarray[i+1] = min(qt, 1000*qsatw(tarray[i+1], pressure/100))
            qlarray[i+1] = max(qt - qvarray[i+1], 0)
        else:
            qt = 0.5 - 0.000075 * ( z - 2045.0 )
            theta = 271 + (z - 2000)**(0.33)
            tarray[i+1] = theta * (pressure / 100000)**(0.286)
            qvarray[i+1] = min(qt, 1000*qsatw(tarray[i+1], pressure/100))
            qlarray[i+1] = max(qt - qvarray[i+1], 0)
    return {"z": zarray, "p": parray, "t": tarray, "qv": qvarray, "ql": qlarray}
#Return a dictionary containing the initial atmospheric profile of the CONTROL simulation

In [ ]:
def get_vapmix(dwpt, p):
    vappres = 6.1078 * np.exp((17.629*(dwpt - 273.16))/(dwpt-35.86))
    vapmix = 0.622 * vappres / (p-vappres)
    return vapmix
#Get water vapor mixing ratio in kg/kg from dewpoint in K and pressure in hPa

In [ ]:
def esatw(t):
    a0, a1, a2, a3, a4, a5, a6, a7, a8 = 6.11239921, 0.443987641, 0.142986287e-1, 0.264847430e-3, 0.302950461e-5, 0.206739458e-7, 0.640689451e-10, -0.952447341e-13,-0.976195544e-15
    dt = max(-80.,t-273.16)
    esatw = a0 + dt*(a1+dt*(a2+dt*(a3+dt*(a4+dt*(a5+dt*(a6+dt*(a7+a8*dt))))))) 
    return esatw
#Return water equilibrium vapor pressure in hPa from temperature in K

In [ ]:
def qsatw(t,p):
    esat = esatw(t)
    qsatw = 0.622 * esat/max(esat,p-esat)
    return qsatw
#Return water vapor equilibrium vapor pressure in hPa from temperature in K

In [ ]:
def esati(tkelv):
    a0,a1,a2,a3,a4,a5,a6,a7,a8 = 6.11147274, 0.503160820, 0.188439774e-1, 0.420895665e-3, 0.615021634e-5,0.602588177e-7, 0.385852041e-9, 0.146898966e-11, 0.252751365e-14
    if(tkelv > 185):
        dt = tkelv-273.16
        esati = a0 + dt*(a1+dt*(a2+dt*(a3+dt*(a4+dt*(a5+dt*(a6+dt*(a7+a8*dt))))))) 
    else:
        dt = max(-100.,tkelv-273.16)
        esati = 0.00763685 + dt*(0.000151069+dt*7.48215e-07)
    return esati

In [ ]:
def find_top(cloud, thresh):
    first_ind = (cloud>thresh).cumsum(dim = "z").argmax(dim = "z")
    previous_ind=first_ind+1
    first_z=cloud.z.isel(z=first_ind)
    previous_z=cloud.z.isel(z=previous_ind)
    first_wc=cloud.isel(z=first_ind)
    previous_wc=cloud.isel(z=previous_ind)
    intermediate_z=(thresh-previous_wc)/(first_wc-previous_wc)*(first_z-previous_z)+previous_z
    return intermediate_z

In [ ]:
def integrate_subset(da, z1s, z2s):
    dz = 10 #vertical grid spacing in our simulations
    tlen = len(da["time"])
    intvals = np.zeros(tlen)
    for i,t in enumerate(da["time"]):
        intvals[i] = da.sel(time = t, z = slice(z1s[i], z2s[i])).integrate(coord = "z")
    return intvals

In [ ]:
with open("/home/b/Ben.Ascher/SAM/SAM6.10.10.LCM/ISDAC/snd") as sndfile_control:
    sndfile_quantities = " ".join(sndfile_control.readline())
    sndhead = sndfile_control.readline()
    day0 = 0.0
    numlvs = int(sndhead[6:9])
    pres0 = float(sndhead[11:17])
    sndzs, sndps, sndthetas, sndts, sndvapmixs, sndus, sndvs = [np.zeros(numlvs) for i in range(7)]
    sndline = sndfile_control.readline()
    sndz0 = sndline[0:9]
    sndtheta = sndline[19:27]
    sndvapmix = sndline[29:37]
    sndu = sndline[37:46]
    sndv = sndline[46:54]
    sndzs[0] = float(sndz0.strip()); sndps[0] = pres0
    sndthetas[0] = float(sndtheta.strip())
    sndvapmixs[0] = float(sndvapmix.strip())
    sndus[0] = float(sndu.strip())
    sndvs[0] = float(sndv.strip())
    t0 = float(sndtheta.strip()) * (pres0/1000) ** (287/1000)
    sndts[0] = t0
    for i in range(1, numlvs):
        sndline = sndfile_control.readline()
        sndz = float(sndline[0:8].strip())
        sndtheta = float(sndline[19:27].strip())
        sndvapmix = float(sndline[29:37].strip())
        sndu = float(sndline[37:46].strip())
        sndv = float(sndline[46:54].strip())
        rhobelow = sndps[i-1] / (sndts[i-1]* (1 + 0.61 * sndvapmixs[i-1]/1000) * 287)
        dpdz = -1*rhobelow*9.81 #dp/dz = -rho * g in hydrostatic balance
        sndp = sndps[i-1] + dpdz * (sndz - sndzs[i-1])
        sndt = sndtheta * (sndp/1000) ** (287/1000)
        sndzs[i] = sndz
        sndps[i] = sndp
        sndthetas[i] = sndtheta
        sndts[i] = sndt
        sndvapmixs[i] = sndvapmix
        sndus[i] = sndu
        sndvs[i] = sndv
    snddf = pd.DataFrame({"z[m]": sndzs, "p[mb]": sndps, "tp[K]": sndthetas, "t[K]": sndts, "q[g/kg]": sndvapmixs, "u[m/s]": sndus, "v[m/s]": sndvs})

In [ ]:
import os
if not os.path.exists("paperfigs"):
    os.mkdir("paperfigs")

In [ ]:
def isdac_ptoz(p0):
    return np.interp(p0, isdac_controlsnd["p"][::-1]/100, isdac_controlsnd["z"][::-1])
def isdac_ztop(z0):
    return np.interp(z0, isdac_controlsnd["z"], isdac_controlsnd["p"]/100)

isdac_controlsnd = isdac_snd_control()
tfig = plt.figure(figsize = (4,2.5), dpi = 200)
skewax = SkewT(fig = tfig, rotation = 45)
dwptarr = get_dwpt(snddf["q[g/kg]"]/1000, snddf["p[mb]"])
control_dwpt = get_dwpt(isdac_controlsnd["qv"]/1000, isdac_controlsnd["p"]/100)
skewax.ax.set_xlim(-20, 0)
skewax.ax.set_ylim(isdac_controlsnd["p"][0]/100, isdac_controlsnd["p"][-1]/100)
skewax.ax.set_ylabel("Pressure (hPa)")
skewax.ax.set_xlabel("Temperature ($\degree C$)")
skewax.plot_dry_adiabats(t0 = np.arange(-70, 20, 10)*units.degC, pressure = np.arange(1100, 600, -10)*units.hPa, linewidth = 0.7, color = "tomato", label = "Dry Adiabats")
skewax.plot_moist_adiabats(t0 = np.arange(-70, 20, 10)*units.degC, pressure = np.arange(1100, 600, -10)*units.hPa, linewidth = 0.7, color = "dodgerblue", label = "Moist Adiabats")
skewax.plot_mixing_lines(mixing_ratio = np.array([1.2, 1.4, 1.6, 1.8])*units("g/kg"), pressure = np.array([1050, 1000, 950, 900])*units.hPa, alpha = 0.4, linewidth = 0.5)
skewax.plot(isdac_controlsnd["p"] * units.Pa, isdac_controlsnd["t"] * units.K, color = "red", linewidth = 1, label = "Temperature")
skewax.plot(isdac_controlsnd["p"] * units.Pa, (control_dwpt) * units.K, color = "blue", linewidth = 1, label = "Dewpoint")
secax = skewax.ax.secondary_yaxis(-0.2, functions = (isdac_ptoz, isdac_ztop))
secax.yaxis.set_major_locator(plt.FixedLocator([0, 400, 800, 1200, 1600, 2000]))
secax.yaxis.set_minor_locator(plt.NullLocator())
secax.yaxis.set_major_formatter(plt.ScalarFormatter())
secax.set_ylabel('Height AMSL (m)')
tfig.legend(loc = "outside lower center", ncols = 2)
tfig.savefig("paperfigs/skewt.png")
plt.close(); del tfig; del secax; del skewax

## Load statistics files for CONTROL

In [ ]:
controlpath = input("Enter the path to the time-height statistics file for the CONTROL simulation: ").strip()
chen_control = xr.open_dataset(controlpath).transpose()

### Ice Number Concentration and QC over time

In [ ]:
chen_lwp = (chen_control["lag_QC"]*chen_control["RHO"]).integrate(coord = "z")
chen_iwp = (chen_control["lag_QI"]*chen_control["RHO"]).integrate(coord = "z")

In [ ]:
print(f"Liquid water path in CONTROL at T=1 hour: {chen_lwp.sel(time = 1/24, method = 'nearest').values:.3f} g / m^2")
print(f"Liquid water path in CONTROL at T=1.7 hours: {chen_lwp.sel(time = 1.7/24, method = 'nearest').values:.3f} g / m^2")
print(f"Liquid water path in CONTROL at T=8 hours: {chen_lwp.sel(time = 1/3, method = 'nearest').values:.3f} g / m^2")

In [ ]:
print(f"Peak ice crystal number concentration in CONTROL: {(chen_control['lag_NIC']*chen_control['RHO']).sel(z = slice(650, 825)).mean(dim = 'z').max(dim = 'time').values*1000:.2f} / L")

### Temperature and Ice Habit

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_ylabel("Altitude AMSL (m)")
    ax.set_xlabel("Elapsed Time (h)")
    ax.contour(24*chen_control["time"], chen_control["z"], chen_control["lag_QC"], levels = [1e-2], colors = "black")
tempmp = ax1.pcolormesh(24*chen_control["time"], chen_control["z"], chen_control["TABS"], vmin = 256, vmax = 266, cmap = "Spectral_r")
habitmp = ax2.pcolormesh(24*chen_control["time"], chen_control["z"], chen_control["lag_iPHI"], vmin = 0, vmax = 0.3, cmap = "Purples")
fig.colorbar(tempmp, ax = [ax1], orientation = "horizontal", aspect = 35, fraction = 0.05, extend = "both", label = "Temperature (K)")
fig.colorbar(habitmp, ax = [ax2], orientation = "horizontal", aspect = 35, fraction = 0.05, extend = "max", label = "Mean Aspect Ratio (-)")
fig.savefig("paperfigs/supplemental_fig1_CONTROL_temp_habit.png")
plt.close()

### Time lags between ice falling below cloud base and precipitation reaching the surface

In [ ]:
sub_startheight_chen = chen_control["z"].where(chen_control["lag_iRH"]>1).min(dim = "z")
min_cloudheight_chen = chen_control["z"].where(chen_control["lag_QC"]>1e-2).min(dim = "z")-10 #One vertical level below cloud base
min_mixheight_chen = chen_control["z"].where(chen_control["TKE"]>0.15).min(dim = "z")

In [ ]:
maxcorr = 0
corrind = 0
for tlen in range(1, len(chen_control["time"])):
    corr = np.correlate(chen_control["lag_NIC"].sel(z = 5).isel(time = slice(tlen, len(chen_control["time"]))),
                        chen_control["lag_NIC"].sel(z = sub_startheight_chen[tlen].values).isel(time = slice(0, len(chen_control["time"])-tlen)))
    if corr > maxcorr:
        maxcorr = corr
        corrind = tlen

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
ax1.plot(24*chen_control["time"], 1000*(chen_control["lag_NIC"]*chen_control["RHO"]).sel(z = 5), color = "blue", label = "Surface Ice Crystal Number Concentration")
ax1.plot(24*chen_control["time"], 1000*(chen_control["lag_NIC"]*chen_control["RHO"]).sel(z = sub_startheight_chen), color = "red", label = "$Z_{sub}$ Ice Crystal Number Concentration")
ax1.set_ylim(0,1)
ax1.legend(fontsize = 7)
ax2.plot(24*chen_control["time"][corrind:len(chen_control["time"])], ((chen_control["lag_NIC"]*chen_control["RHO"]).sel(z = 5).isel(time = slice(corrind, len(chen_control["time"]))).values/
                                   (chen_control["lag_NIC"]*chen_control["RHO"]).sel(z = sub_startheight_chen).isel(time = slice(0, len(chen_control["time"])-corrind))).values, label = "Surface Ice Number Mixing Ratio Fraction")
ax2.legend(fontsize = 7)
ax2.set_ylim(0,1)
ax2.set_xlim(0,8)
ax1.set_ylabel("Ice Crystal Number Concentration ($\# \ L^{-1}$)")
ax1.set_xlabel("Elapsed Time (h)")
ax2.set_xlabel("Surface ICNC Fraction")
fig.savefig("paperfigs/supplemental_fig2_control_srficenummix_timeseries.png")
plt.close()

### Desiccation and IN Concentration

In [ ]:
from scipy.ndimage import gaussian_filter
from palettable.cubehelix import red_16
incmap = colors.LinearSegmentedColormap.from_list(name = "incmap", colors = red_16.get_mpl_colormap().reversed()(np.linspace(0, 0.75, 16)))
truncblues = colors.LinearSegmentedColormap.from_list(name = "Blues_trunc", colors = plt.get_cmap("Blues")(np.linspace(0.2, 1, 128)))
fig, ax1 = plt.subplots(1,1, figsize = (3,2), dpi = 250)
ax1.set_ylim(0, 1000)
ax1.axvline(1, color = "black", linestyle = ":")
inncmp = ax1.pcolormesh(24*chen_control["time"], chen_control["z"], 1000*(chen_control["lag_NINC"]*chen_control["RHO"]), cmap = incmap, vmin = 0, vmax = 2.5)
ax1.contour(24*chen_control["time"], chen_control["z"], chen_control["lag_QC"], levels = [1e-2], colors = "black", linestyles = "--")
nsubmp = ax1.contour(24*chen_control["time"], chen_control["z"], gaussian_filter(1000*(chen_control["lag_NSUB"]*chen_control["RHO"]).values, sigma = [3,3], mode = "nearest", axes = [0,1]), levels = [0.5e-4, 1e-4, 1.5e-4, 2e-4, 2.5e-4], cmap = truncblues)
cbar1 = fig.colorbar(inncmp, fraction = 0.05, aspect = 25, ax = ax1, orientation = "horizontal", anchor = (0.5, 1.0), extend = "max")
cbar1.set_label(label = "IN Number Concentration ($\# \ L^{-1}$)", fontsize = 6.5)
ax1.set_ylabel("Altitude (m AMSL)")
ax1.set_xlabel("Elapsed Time (h)")
fig.savefig("paperfigs/control_innc_nsub.png")
plt.close(); del fig; del ax1; del cbar1

### Mixed-layer QV comparison

In [ ]:
norm_mixheights = np.linspace(0, 1, 101)
cldbasevals_chen = min_cloudheight_chen.values[np.logical_not(np.isnan(min_mixheight_chen.values))];
mixhghtvals_chen = min_mixheight_chen.values[np.logical_not(np.isnan(min_mixheight_chen.values))]
normheights_chen = mixhghtvals_chen[None,:] + norm_mixheights[:,None]*(mixhghtvals_chen[None,:]+cldbasevals_chen[None,:])
time_offset = len(chen_control["time"]) - len(mixhghtvals_chen)
normheightda_chen = xr.DataArray(coords = {"z": (("norm_height", "time"), normheights_chen), "norm_height": norm_mixheights, "time": chen_control["time"][time_offset:]}, dims = ["norm_height", "time"])
chen_mixqv = chen_control["QV"].isel(time = slice(time_offset, len(chen_control["time"]))).interp(z = normheightda_chen["z"])

In [ ]:
mean_mixqv_chen = chen_control["QV"].where((chen_control["QV"]>1.4)*(chen_control["QV"]<1.6)).sel(time = slice(1/12, 1/3)).mean()

### Desiccation rates comparison

In [ ]:
mean_nsub_chen = chen_control["lag_NSUB"].sel(z = slice(0, 400), time = slice(1/12, 1/3)).mean()
print(f"Average desiccation rate between hours 2 and 8 in Chen_CONTROL: {mean_nsub_chen:.3e} particles/m3/s")

## Comparisons between CONTROL and DRY

In [ ]:
drypath = input("Enter the path to the time-height statistics file for the DRY simulation: ").strip()
chen_dry = xr.open_dataset(drypath).transpose()

In [ ]:
chen_dry_lwp = (chen_dry["lag_QC"]*chen_dry["RHO"]).integrate(coord = "z")

print(f"Chen_CONTROL LWP at T=1 hour: {chen_lwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Chen_DRY LWP at T=1 hour: {chen_dry_lwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Absolute difference: {chen_dry_lwp.sel(time = 1/24, method = 'nearest') - chen_lwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Relative difference: {(chen_dry_lwp.sel(time = 1/24, method = 'nearest') - chen_lwp.sel(time = 1/24, method = 'nearest'))/chen_lwp.sel(time = 1/24, method = 'nearest')*100:.3f}%")
chen_lwp = chen_lwp.sel(time = slice(1/12, 1/3)).mean()
chen_dry_lwp = chen_dry_lwp.sel(time = slice(1/12, 1/3)).mean()

In [ ]:
chen_dry_iwp = (chen_dry["lag_QI"]*chen_dry["RHO"]).integrate(coord = "z")

print(f"Chen_CONTROL IWP at T=1 hour: {chen_iwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Chen_DRY IWP at T=1 hour: {chen_dry_iwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Absolute difference: {chen_dry_iwp.sel(time = 1/24, method = 'nearest') - chen_iwp.sel(time = 1/24, method = 'nearest')} g/kg")
print(f"Relative difference: {(chen_dry_iwp.sel(time = 1/24, method = 'nearest') - chen_iwp.sel(time = 1/24, method = 'nearest'))/chen_iwp.sel(time = 1/24, method = 'nearest')*100:.3f}%")
chen_iwp = chen_iwp.sel(time = slice(1/12, 1/3)).mean()
chen_dry_iwp = chen_dry_iwp.sel(time = slice(1/12, 1/3)).mean()

In [ ]:
print(f"Average LWP in CHEN_CONTROL between hours 2 and 8: {chen_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_DRY between hours 2 and 8: {chen_dry_lwp:.3f} g/m^2")
print(f"Relative difference: {(chen_dry_lwp - chen_lwp)/chen_lwp:.3%}")

In [ ]:
print(f"Average cloud-top radiative cooling in Chen_CONTROL between hours 2 and 8: {chen_control['RADQRLW'].sel(time = slice(1/12, 1/3), z = slice(775, 835)).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_DRY between hours 2 and 8: {chen_dry['RADQRLW'].sel(time = slice(1/12, 1/3), z = slice(775, 835)).mean():.3f} K/day")

In [ ]:
mean_mixqv_chendry = chen_dry["QV"].where((chen_dry["QV"]>1.4)*(chen_dry["QV"]<1.6)).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference between DRY and CONTROL mixed-layer mixing ratio: {(mean_mixqv_chendry - mean_mixqv_chen)/mean_mixqv_chen*100}%")

In [ ]:
sub_startheight_chendry = chen_dry["z"].where(chen_dry["lag_iRH"]>1).min(dim = "z")

### Radius Comparison of CONTROL and DRY

In [ ]:
from palettable.cmocean.sequential import Dense_10
fig, (ax11, ax12, ax13) = plt.subplots(1,3, figsize = (7, 2.5), dpi = 250, sharex = True, sharey = True)
axlabels = ["(a)", "(b)", "(c)"]
# axtitles = ["CONTROL", "DRY", "CONTROL - DRY"]
l,w,b,h = 0,0.1,0.9,0.1
for i,ax in enumerate((ax11, ax12, ax13)):
    ax = add_subplotlabel(ax, axlabels[i], l, w, b, h)
    ax.set_ylim(0, 950)
    ax.set_xlim(0, 8)
    ax.axvline(1, color = "black", linestyle = ":")
ax11.set_title("CONTROL")
ax12.set_title("DRY")
ax13.set_title("CONTROL - DRY")
ax11.tick_params(left = True, labelleft = True, bottom = False, labelbottom = False)
ax12.tick_params(left = False, labelleft = False, bottom = True, labelbottom = True)
ax13.tick_params(left = False, labelleft = False, bottom = False, labelbottom = False)
ax11.set_ylabel("Altitude (m AMSL)")
ax12.set_xlabel("Elapsed Time (h)")
radmp = ax11.pcolormesh(24*chen_control["time"], chen_control["z"], chen_control["lag_iR"], vmin = 0, vmax = 1200, cmap = Dense_10.get_mpl_colormap())
ax11.contour(24*chen_control["time"], chen_control["z"], chen_control["lag_QC"], levels = [1e-2], colors = "black")
ax11.plot(24*chen_control["time"], sub_startheight_chen, color = "limegreen")
ax12.pcolormesh(24*chen_control["time"], chen_control["z"], chen_dry["lag_iR"], vmin = 0, vmax = 1200, cmap = Dense_10.get_mpl_colormap())
ax12.contour(24*chen_control["time"], chen_control["z"], chen_dry["lag_QC"], levels = [1e-2], colors = "black")
ax12.plot(24*chen_control["time"], sub_startheight_chendry, color = "limegreen")
raddiffmp = ax13.pcolormesh(24*chen_control["time"], chen_control["z"], chen_control["lag_iR"] - chen_dry["lag_iR"], vmin = -400, vmax = 400, cmap = "bwr_r")
fig.colorbar(radmp, ax = [ax11, ax12], orientation = "horizontal", aspect = 35, fraction = 0.1, label = "Ice Mean Radius ($\mu m$)", extend = "max")
fig.colorbar(raddiffmp, ax = [ax13], orientation = "horizontal", aspect = 18, fraction = 0.1, label = "Difference in Mean Radius ($\mu m$)", extend = "both")
fig.savefig("paperfigs/control_dry_radiuscomp.png")
plt.close()

### Deposition Rate Per Particle Comparison

In [ ]:
fig, (ax11, ax12, ax13) = plt.subplots(1, 3, figsize = (7,2.5), dpi = 250)
axlabels = ["(a)", "(b)", "(c)"]
l,w,b,h = 0, 0.1, 0.9, 0.1
for i, ax in enumerate((ax11, ax12, ax13)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_ylim(0,1000)
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
ax11.set_title("CONTROL")
ax12.set_title("DRY")
ax13.set_title("CONTROL - DRY")
ax11.tick_params(left = True, labelleft = True, bottom = False, labelbottom = False)
ax12.tick_params(left = False, labelleft = False, bottom = True, labelbottom = True)
ax13.tick_params(left = False, labelleft = False, bottom = False, labelbottom = False)
ax11.set_ylabel("Altitude (m AMSL)")
ax12.set_xlabel("Elapsed Time (h)")
netdepmp = ax11.pcolormesh(24*chen_control["time"], chen_control["z"], (chen_control["lag_DEP"]+chen_control["lag_SUB"])/(10**6*chen_control["lag_NIC"]), vmin = -1.5e-7, vmax = 1.5e-7, cmap = "BrBG")
ax11.contour(24*chen_control["time"], chen_control["z"], chen_control["lag_QC"], levels = [1e-2], colors = "black")
ax12.pcolormesh(24*chen_control["time"], chen_control["z"], (chen_dry["lag_DEP"]+chen_dry["lag_SUB"])/(10**6*chen_dry["lag_NIC"]), vmin = -1.5e-7, vmax = 1.5e-7, cmap = "BrBG")
ax12.contour(24*chen_control["time"], chen_control["z"], chen_dry["lag_QC"], levels = [1e-2], colors = "black")
depdiffmp = ax13.pcolormesh(24*chen_control["time"], chen_control["z"], ((chen_control["lag_DEP"]+chen_control["lag_SUB"])/(10**6*chen_control["lag_NIC"]) - (chen_dry["lag_DEP"]+chen_dry["lag_SUB"])/(10**6*chen_dry["lag_NIC"])), vmin = -0.5e-7, vmax = 0.5e-7, cmap = "RdBu")
cbar1 = fig.colorbar(netdepmp, ax = [ax11, ax12], orientation = "horizontal", aspect = 35, fraction = 0.1, extend = "both"); cbar1.set_label("Net Deposition Rate Per Particle ($g \ particle^{-1} \ s^{-1}$)")
cbar2 = fig.colorbar(depdiffmp, ax = [ax13], orientation = "horizontal", aspect = 18, fraction = 0.1, extend = "both"); cbar2.set_label("Difference in Net Deposition \n Rate Per Particle ($g \ particle^{-1} \ s^{-1}$)")
fig.savefig("paperfigs/netdepcomp_paper.png")
plt.close()
del fig; del ax11; del ax12; del ax13
del cbar1; del cbar2

### Comparison of IN Number Concentration Between CONTROL and DRY

In [ ]:
from palettable.cubehelix import red_16
from palettable.cmocean.diverging import Curl_11
incmap = colors.LinearSegmentedColormap.from_list(name = "incmap", colors = red_16.get_mpl_colormap().reversed()(np.linspace(0, 0.75, 16)))
truncblues = colors.LinearSegmentedColormap.from_list(name = "Blues_trunc", colors = plt.get_cmap("Blues_r")(np.linspace(0, 0.8, 128)))
print(incmap)
fig, (ax11, ax12, ax13) = plt.subplots(1, 3, figsize = (7,2.5), dpi = 250)
axlabels = ["(a)", "(b)", "(c)"]
l,w,b,h = 0, 0.1, 0.9, 0.1
for i, ax in enumerate((ax11, ax12, ax13)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_ylim(0,1000)
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
ax11.set_title("CONTROL")
ax12.set_title("DRY")
ax13.set_title("CONTROL - DRY")
ax11.tick_params(left = True, labelleft = True, bottom = False, labelbottom = False)
ax12.tick_params(left = False, labelleft = True, bottom = True, labelbottom = False)
ax13.tick_params(left = False, labelleft = False, bottom = False, labelbottom = False)
ax11.set_ylabel("Altitude (m AMSL)")
ax12.set_xlabel("Elapsed Time (h)")
inmp = ax11.pcolormesh(24*chen_control["time"], chen_control["z"], (1000*chen_control["lag_NINC"]*chen_control["RHO"]), vmin = 0, vmax = 2.5, cmap = incmap)
ax11.contour(24*chen_control["time"].sel(time = slice(1/24, 1/3)), chen_control["z"], gaussian_filter(chen_control["lag_NUC"].sel(time = slice(1/24, 1/3)).values, (3,5), mode = "nearest"), cmap = truncblues, levels = [2e-7, 4e-7, 6e-7, 8e-7, 1e-6], norm = "log")
ax12.pcolormesh(24*chen_control["time"], chen_control["z"], 1000*chen_dry["lag_NINC"]*chen_dry["RHO"], vmin = 0, vmax = 2.5, cmap = incmap)
ax12.contour(24*chen_control["time"].sel(time = slice(1/24, 1/3)), chen_control["z"], gaussian_filter(chen_dry["lag_NUC"].sel(time = slice(1/24, 1/3)).values, (3,5), mode = "nearest"), cmap = truncblues, levels = [2e-7, 4e-7, 6e-7, 8e-7, 1e-6], norm = "log")
indiffmp = ax13.pcolormesh(24*chen_control["time"], chen_control["z"], 1000*(chen_control["lag_NINC"]*chen_control["RHO"] - chen_dry["lag_NINC"]*chen_dry["RHO"]), vmin = -0.5, vmax = 0.5, cmap = "RdBu_r")
cbar1 = fig.colorbar(inmp, ax = [ax11, ax12], orientation = "horizontal", aspect = 35, fraction = 0.1, extend = "max"); cbar1.set_label("IN Number Concentration ($\# \ L^{-1}$)")
cbar2 = fig.colorbar(indiffmp, ax = [ax13], orientation = "horizontal", aspect = 18, fraction = 0.1, extend = "both"); cbar2.set_label("Difference in IN Number Concentration ($\# \ L^{-1}$)")
fig.savefig("paperfigs/innummixcomp_paper.png")
plt.close(); del fig; del ax11; del ax12; del ax13
del cbar1; del cbar2

In [ ]:
print(f"Average depth of ice-subsaturated region in Chen_CONTROL: {sub_startheight_chen.sel(time = slice(1/12, 1/3)).mean() - 10:.3f} m")
print(f"Average depth of ice-subsaturated region in Chen_DRY: {sub_startheight_chendry.sel(time = slice(1/12, 1/3)).mean() - 10:.3f} m")

In [ ]:
print(f"Average relative humidity w.r.t. ice in ice-subsaturated region in Chen_CONTROL: {chen_control['lag_iRH'].sel(z = slice(0,825)).where(chen_control['lag_iRH']<1).sel(time = slice(1/12, 1/3)).mean():.3%}")
print(f"Average relative humidity w.r.t. ice in ice-subsaturated region in Chen_DRY: {chen_dry['lag_iRH'].sel(z = slice(0,825)).where(chen_dry['lag_iRH']<1).sel(time = slice(1/12, 1/3)).mean():.3%}")

In [ ]:
iwv_chenda = (((1-chen_control["lag_iRH"])*chen_control["QV"])*chen_control["RHO"]).sel(z = slice(0, 700)).where(chen_control["lag_iRH"]<1)
iwv_chen = np.asarray([np.trapz(iwv_chenda.sel(time = t).values[np.logical_not(np.isnan(iwv_chenda.sel(time = t).values))], iwv_chenda["z"].values[np.logical_not(np.isnan(iwv_chenda.sel(time = t).values))]) for t in iwv_chenda["time"]])
iwv_chendryda = (((1-chen_dry["lag_iRH"])*chen_dry["QV"])*chen_dry["RHO"]).sel(z = slice(0, 700)).where(chen_dry["lag_iRH"]<1)
iwv_chendry = np.asarray([np.trapz(iwv_chendryda.sel(time = t).values[np.logical_not(np.isnan(iwv_chendryda.sel(time = t).values))], iwv_chendryda["z"].values[np.logical_not(np.isnan(iwv_chendryda.sel(time = t).values))]) for t in iwv_chendryda["time"]])

In [ ]:
print(f"Average integrated water vapor deficit w.r.t. ice between hours 2 and 8 in Chen_CONTROL: {iwv_chen[240:].mean():.3f} g/m^2")
print(f"Average integrated water vapor deficit w.r.t. ice between hours 2 and 8 in Chen_DRY: {iwv_chendry[240:].mean():.3f} g/m^2")
print(f"Relative difference between Chen_DRY and Chen_CONTROL: {((iwv_chendry[240:]-iwv_chen[240:])/iwv_chen[240:]).mean():.3%}")

In [ ]:
chen_subfraction = chen_control["lag_NIC"].sel(z=5, method = "nearest") / chen_control["lag_NIC"].sel(z = sub_startheight_chen)
chen_dry_subfraction = chen_dry["lag_NIC"].sel(z=5, method = "nearest") / chen_dry["lag_NIC"].sel(z = sub_startheight_chendry)

print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_CONTROL: {chen_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")
print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_DRY: {chen_dry_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")

## Comparisons between CONTROL and INFLUX

In [ ]:
influxpath = input("Enter the path to the time-height statistics file for the INFLUX simulation: ").strip()
chen_influx = xr.open_dataset(influxpath).transpose()

In [ ]:
chen_influx_lwp = (chen_influx["lag_QC"]*chen_influx["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average LWP in CHEN_CONTROL between hours 2 and 8: {chen_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_INFLUX between hours 2 and 8: {chen_influx_lwp:.3f} g/m^2")
print(f"Relative difference: {(chen_influx_lwp - chen_lwp)/chen_lwp:.3%}")

In [ ]:
chen_influx_iwp = (chen_influx["lag_QI"]*chen_influx["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average iwp in CHEN_CONTROL between hours 2 and 8: {chen_iwp:.3f} g/m^2")
print(f"Average iwp in CHEN_INFLUX between hours 2 and 8: {chen_influx_iwp:.3f} g/m^2")
print(f"Relative difference: {(chen_influx_iwp - chen_iwp)/chen_iwp:.3%}")

In [ ]:
chen_depper = (chen_control["lag_DEP"]/chen_control["lag_NIC"]).where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_influx_depper = (chen_influx["lag_DEP"]/chen_influx["lag_NIC"]).where(chen_influx["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in CHEN_INFLUX between hours 2 and 8: {chen_influx_depper:.4f} ug/particle/s")
print(f"Relative difference: {(chen_influx_depper - chen_depper)/chen_depper:.3%}")

In [ ]:
chen_nic = (chen_control["lag_NIC"]*chen_control["RHO"]*1000).where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_influx_nic = (chen_influx["lag_NIC"]*chen_influx["RHO"]*1000).where(chen_influx["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud ice crystal number mixing ratio in Chen_CONTROL between hours 2 and 8: {chen_nic:.4f} particles/L")
print(f"Average in-cloud ice crystal number mixing ratio in CHEN_INFLUX between hours 2 and 8: {chen_influx_nic:.4f} particles/L")
print(f"Relative difference: {(chen_influx_nic - chen_nic)/chen_nic:.3%}")

In [ ]:
chen_dep = (chen_control["lag_DEP"]).where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_influx_dep = (chen_influx["lag_DEP"]).where(chen_influx["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud total deposition rate in Chen_CONTROL between hours 2 and 8: {chen_dep:.3e} g/kg/s")
print(f"Average in-cloud total deposition rate in CHEN_INFLUX between hours 2 and 8: {chen_influx_dep:.3e} g/kg/s")
print(f"Relative difference: {(chen_influx_dep - chen_dep)/chen_dep:.3%}")

In [ ]:
chen_cnd = (chen_control["lag_CND"]).where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_influx_cnd = (chen_influx["lag_CND"]).where(chen_influx["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud total condensation rate in Chen_CONTROL between hours 2 and 8: {chen_cnd:.3e} g/kg/s")
print(f"Average in-cloud total condensation rate in CHEN_INFLUX between hours 2 and 8: {chen_influx_cnd:.3e} g/kg/s")
print(f"Relative difference: {(chen_influx_cnd - chen_cnd)/chen_cnd:.3%}")

In [ ]:
print(f"Average cloud-top radiative cooling in Chen_CONTROL between hours 2 and 8: {chen_control['RADQRLW'].sel(time = slice(1/12, 1/3), z = slice(775, 835)).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_INFLUX between hours 2 and 8: {chen_influx['RADQRLW'].sel(time = slice(1/12, 1/3), z = slice(775, 835)).mean():.3f} K/day")
print(f"Relative difference: {((chen_influx['RADQRLW']-chen_control['RADQRLW'])/chen_control['RADQRLW']).sel(time = slice(1/12, 1/3), z = slice(775, 835)).mean():.3%}")

In [ ]:
chen_influx_tkepath = chen_influx["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
chen_tkepath = chen_control["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Mean TKE path in Chen_CONTROL between hours 2 and 8: {chen_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_INFLUX between hours 2 and 8: {chen_influx_tkepath:.3f} m^3 s^-2")
print(f'''
Relative difference in mean TKE path between Chen_CONTROL and Chen_INFLUX between hours 2 and 8: 
{(chen_influx_tkepath - chen_tkepath)/chen_tkepath:.3%}''')

## Comparisons between CONTROL and WEAKINV

In [ ]:
weakinvpath = input("Enter the path to the time-height statistics file for the WEAKINV simulation: ").strip()
chen_weakinv = xr.open_dataset(weakinvpath).transpose()
sub_startheight_weakinv = chen_weakinv["z"].where(chen_weakinv["lag_iRH"]>1).min(dim = "z")

In [ ]:
chen_weakinv_lwp = (chen_weakinv["lag_QC"]*chen_weakinv["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average LWP in Chen_CONTROL between hours 2 and 8: {chen_lwp:.3f} g/kg")
print(f"Average LWP in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_lwp:.3f} g/kg")
print(f"Relative difference: {(chen_weakinv_lwp - chen_lwp)/chen_lwp:.3%}")

qcmean_control = chen_control["lag_QC"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
qcmean_weakinv = chen_weakinv["lag_QC"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud liquid water mixing ratio between WEAKINV and CONTROL between hours 2 and 8: {(qcmean_weakinv - qcmean_control)/qcmean_control:.3%}")

condmean_control = chen_control["lag_CND"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
condmean_weakinv = chen_weakinv["lag_CND"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud condensation rate ratio between WEAKINV and CONTROL between hours 2 and 8: {(condmean_weakinv - condmean_control)/condmean_control:.3%}")

evapmean_control = chen_control["lag_EVP"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
evapmean_weakinv = chen_weakinv["lag_EVP"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud evaporation rate between WEAKINV and CONTROL between hours 2 and 8: {(evapmean_weakinv - evapmean_control)/evapmean_control:.3%}")

In [ ]:
chen_weakinv_iwp = (chen_weakinv["lag_QI"]*chen_weakinv["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average IWP in Chen_CONTROL between hours 2 and 8: {chen_iwp:.3f} g/m^2")
print(f"Average IWP in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_iwp:.3f} g/m^2")
print(f"Relative difference: {(chen_weakinv_iwp - chen_iwp)/chen_iwp:.3%}")

qimean_control = chen_control["lag_QI"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
qimean_weakinv = chen_weakinv["lag_QI"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud ice water mixing ratio between WEAKINV and CONTROL between hours 2 and 8: {(qimean_weakinv - qimean_control)/qimean_control:.3%}")

depmean_control = chen_control["lag_DEP"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
depmean_weakinv = chen_weakinv["lag_DEP"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud deposition rate between WEAKINV and CONTROL between hours 2 and 8: {(depmean_weakinv - depmean_control)/depmean_control:.3%}")

In [ ]:
chen_weakinv_depper = (chen_weakinv["lag_DEP"]/chen_weakinv["lag_NIC"]).where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_depper:.4f} ug/particle/s")
print(f"Relative difference: {(chen_weakinv_depper - chen_depper)/chen_depper:.3%}")


In [ ]:
chen_weakinv_nic = (chen_weakinv["lag_NIC"]*chen_weakinv["RHO"]*1000).where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud ice-crystal number concentration in Chen_CONTROL between hours 2 and 8: {chen_nic:.4f} particles/L")
print(f"Average in-cloud ice-crystal number concentration in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_nic:.4f} particles/L")

print(f"Relative difference: {(chen_weakinv_nic - chen_nic)/chen_nic:.3%}")


In [ ]:
chen_weakinv_dep = (chen_weakinv["lag_DEP"]).where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average total in-cloud deposition rate in Chen_CONTROL between hours 2 and 8: {chen_dep:.3e} g/kg/s")
print(f"Average total in-cloud deposition rate in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_dep:.3e} g/kg/s")

print(f"Relative difference: {(chen_weakinv_dep - chen_dep)/chen_dep:.3%}")


In [ ]:
chen_weakinv_cnd = (chen_weakinv["lag_CND"]).where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average total in-cloud condensation rate in Chen_CONTROL between hours 2 and 8: {chen_cnd:.3e} g/kg/s")
print(f"Average total in-cloud condensation rate in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_cnd:.3e} g/kg/s")

print(f"Relative difference: {(chen_weakinv_cnd - chen_cnd)/chen_cnd:.3%}")


In [ ]:
cloudtop_control = find_top(chen_control["lag_QC"], 1e-2).values
topheights_control = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_control[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_weakinv = find_top(chen_weakinv["lag_QC"], 1e-2).values
topheights_weakinv = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_weakinv[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
print(f"Average Cloud-top longwave cooling rate in Chen_CONTROL between hours 2 and 8: {chen_control['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_control['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average Cloud-top longwave cooling rate in Chen_WEAKINV between hours 2 and 8: {chen_weakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_weakinv['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")

In [ ]:
cloudbase_control = find_top(chen_control["lag_QC"][::-1], 1e-2);
cloudbase_weakinv = find_top(chen_weakinv["lag_QC"][::-1], 1e-2)
cloudthick_control = (cloudtop_control - cloudbase_control).sel(time = slice(1/12, 1/3)).mean()
cloudthick_weakinv = (cloudtop_weakinv - cloudbase_weakinv).sel(time = slice(1/12, 1/3)).mean()
print(f"Average cloud thickness in Chen_CONTROL between hours 2 and 8: {cloudthick_control:.2f} m")
print(f"Average cloud thickness in Chen_WEAKINV between hours 2 and 8: {cloudthick_weakinv:.2f} m")
print(f"Relative difference between Chen_CONTROL and Chen_WEAKINV: {(cloudthick_weakinv - cloudthick_control)/cloudthick_control:.3%}")

### Nucleation rates comparision

In [ ]:
mean_nucrate_control = chen_control["lag_NUC"].where(chen_control["lag_NUC"]>0).sel(time = slice(1/12, 1/3)).mean()
mean_nucrate_weakinv = chen_weakinv["lag_NUC"].where(chen_weakinv["lag_NUC"]>0).sel(time = slice(1/12, 1/3)).mean()
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in Chen_CONTROL: {mean_nucrate_control:.3e} mg/kg/s")
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in Chen_WEAKINV: {mean_nucrate_weakinv:.3e} mg/kg/s")
print(f"Relative difference between Chen_CONTROl and Chen_WEAKINV: {(mean_nucrate_weakinv - mean_nucrate_control)/mean_nucrate_control:.3%}")

vertint_nuc_control = (chen_control["lag_NUC"]*chen_control["RHO"]).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
vertint_nuc_weakinv = (chen_weakinv["lag_NUC"]*chen_weakinv["RHO"]).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
print(f"Vertically integrated nucleation rate between hours 2 and 8 in Chen_CONTROL: {vertint_nuc_control:.3e} mg/m^2/s")
print(f"Vertically integrated nucleation rate between hours 2 and 8 in Chen_WEAKINV: {vertint_nuc_weakinv:.3e} mg/m^2/s")
print(f"Relative difference: {(vertint_nuc_weakinv - vertint_nuc_control)/vertint_nuc_weakinv:.3%}")
print(f"Relative difference in boundary layer height between WEAKINV and CONTROL: {(cloudtop_weakinv[240:960].mean() - cloudtop_control[240:960].mean())/cloudtop_control[240:960].mean():.3%}")

subcldin_control = (10**6*chen_control["lag_NINC"]*chen_control["RHO"]).sel(z = slice(0,700)).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
subcldin_weakinv = (10**6*chen_weakinv["lag_NINC"]*chen_weakinv["RHO"]).sel(z = slice(0,700)).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()

print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_CONTROL: {subcldin_control:.3e} particles/m^2")
print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_CONTROL: {subcldin_weakinv:.3e} particles/m^2")
print(f"Relative difference: {(subcldin_weakinv - subcldin_control)/subcldin_control:.3%}")
print(f"Relative difference in cloud base height between WEAKINV and CONTROL: {(cloudbase_weakinv[240:960].mean() - cloudbase_control[240:960].mean())/cloudbase_control[240:960].mean():.3%}")

### TKE Comparison

In [ ]:
chen_weakinv_tkepath = chen_weakinv["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Mean TKE path Chen_CONTROL between hours 2 and 8: {chen_tkepath:.3f} g/kg/s")
print(f"Average tke path in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_tkepath:.3f} g/kg/s")

print(f"Relative difference: {(chen_weakinv_tkepath - chen_tkepath)/chen_tkepath:.3%}")

tkemean_control = chen_control["TKE"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
tkemean_weakinv = chen_weakinv["TKE"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Mean in-cloud TKE in Chen_CONTROL: {tkemean_control:.3f} m^2/s^2")
print(f"Mean in-cloud TKE Chen_WEAKINV: {tkemean_weakinv:.3f} m^2/s^2")
print(f"Relative difference: {(tkemean_weakinv - tkemean_control)/tkemean_control:.3%}")

w2mean_control = chen_control["W2"].where(chen_control["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
w2mean_weakinv = chen_weakinv["W2"].where(chen_weakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Mean in-cloud vertical velocity variance in Chen_CONTROL: {w2mean_control:.3f} m^2/s^2")
print(f"Mean in-cloud vertical velocity variance in Chen_WEAKINV: {w2mean_weakinv:.3f} m^2/s^2")
print(f"Relative difference: {(w2mean_weakinv - w2mean_control)/w2mean_control:.3%}")

## Comparisons between DRY, WEAKINV, and DRY_WEAKINV

In [ ]:
dryweakinvpath = input("Enter the path to the time-height statistics file for the DRY_WEAKINV simulation: ").strip()
chen_dryweakinv = xr.open_dataset(dryweakinvpath).transpose()
sub_startheight_dryweakinv = chen_dryweakinv["z"].where(chen_dryweakinv["lag_iRH"]>1).min(dim = "z")

In [ ]:
chen_dryweakinv_lwp = (chen_dryweakinv["lag_QC"]*chen_dryweakinv["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average LWP in Chen_CONTROL between hours 2 and 8: {chen_lwp:.3f} g/kg")
print(f"Average LWP in Chen_DRY between hours 2 and 8: {chen_dry_lwp:.3f} g/kg")
print(f"Average LWP in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_lwp:.3f} g/kg")
print(f"Average LWP in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_lwp:.3f} g/kg")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_lwp - chen_lwp)/chen_lwp:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(chen_dryweakinv_lwp - chen_dry_lwp)/chen_dry_lwp:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_lwp - chen_weakinv_lwp)/chen_weakinv_lwp:.3%}")

In [ ]:
chen_dryweakinv_iwp = (chen_dryweakinv["lag_QI"]*chen_dryweakinv["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average IWP in Chen_CONTROL between hours 2 and 8: {chen_iwp:.3f} g/kg")
print(f"Average IWP in Chen_DRY between hours 2 and 8: {chen_dry_iwp:.3f} g/kg")
print(f"Average IWP in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_iwp:.3f} g/kg")
print(f"Average IWP in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_iwp:.3f} g/kg")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_iwp - chen_iwp)/chen_iwp:.3%}")
print(f"Relative difference between DRYL and DRY_WEAKINV: {(chen_dryweakinv_iwp - chen_dry_iwp)/chen_dry_iwp:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_iwp - chen_weakinv_iwp)/chen_weakinv_iwp:.3%}")

pulseiwp_control = (chen_control['lag_QI']*chen_control['RHO']).sel(time = slice(1/24, 1/12)).integrate(coord = 'z').max(dim = 'time')
pulseiwp_dry = (chen_dry['lag_QI']*chen_dry['RHO']).sel(time = slice(1/24, 1/12)).integrate(coord = 'z').max(dim = 'time')
pulseiwp_weakinv = (chen_weakinv['lag_QI']*chen_weakinv['RHO']).sel(time = slice(1/24, 1/12)).integrate(coord = 'z').max(dim = 'time')
pulseiwp_dryweakinv = (chen_dryweakinv['lag_QI']*chen_dryweakinv['RHO']).sel(time = slice(1/24, 1/12)).integrate(coord = 'z').max(dim = 'time')
print(f"Peak IWP in 'pulse' in DRY_WEAKINV: {pulseiwp_dryweakinv:.3f} g/m^2")
print(f"Relative difference in 'pulse' IWP between CONTROL and DRY_WEAKINV: {(pulseiwp_dryweakinv - pulseiwp_control)/pulseiwp_control:.1%}")
print(f"Relative difference in 'pulse' IWP between DRY and DRY_WEAKINV: {(pulseiwp_dryweakinv - pulseiwp_dry)/pulseiwp_dry:.1%}")
print(f"Relative difference in 'pulse' IWP between WEAKINV and DRY_WEAKINV: {(pulseiwp_dryweakinv - pulseiwp_weakinv)/pulseiwp_weakinv:.1%}")
#Here, what we're referring to as "pulse" IWP is the IWP of the initial phase of ice  nucleation and formation between hours 1 and 2.5

qimean_dry = chen_dry["lag_QI"].where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
qimean_dryweakinv = chen_dryweakinv["lag_QI"].where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud ice water mixing ratio between DRY_WEAKINV and CONTROL between hours 2 and 8: {(qimean_dryweakinv - qimean_control)/qimean_control:.3%}")
print(f"Relative difference in in-cloud ice water mixing ratio between DRY_WEAKINV and DRY between hours 2 and 8: {(qimean_dryweakinv - qimean_dry)/qimean_dry:.3%}")
print(f"Relative difference in in-cloud ice water mixing ratio between DRY_WEAKINV and WEAKINV between hours 2 and 8: {(qimean_dryweakinv - qimean_weakinv)/qimean_weakinv:.3%}")

depmean_dry = chen_dry["lag_DEP"].where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
depmean_dryweakinv = chen_dryweakinv["lag_DEP"].where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Relative difference in in-cloud deposition rate ratio between DRY_WEAKINV and CONTROL between hours 2 and 8: {(depmean_dryweakinv - depmean_control)/depmean_control:.3%}")
print(f"Relative difference in in-cloud deposition rate ratio between DRY_WEAKINV and DRY between hours 2 and 8: {(depmean_dryweakinv - depmean_dry)/depmean_dry:.3%}")
print(f"Relative difference in in-cloud deposition rate ratio between DRY_WEAKINV and WEAKINV between hours 2 and 8: {(depmean_dryweakinv - depmean_weakinv)/depmean_weakinv:.3%}")

In [ ]:
chen_dry_depper = (chen_dry["lag_DEP"]/chen_dry["lag_NIC"]).where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_dryweakinv_depper = (chen_dryweakinv["lag_DEP"]/chen_dryweakinv["lag_NIC"]).where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_dry_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_weakinv_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_WEAKINV between hours 2 and 8: {chen_dryweakinv_depper:.4f} ug/particle/s")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_depper - chen_depper)/chen_depper:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(chen_dryweakinv_depper - chen_dry_depper)/chen_dry_depper:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_depper - chen_weakinv_depper)/chen_weakinv_depper:.3%}")

In [ ]:
chen_dry_nic = (chen_dry["lag_NIC"]*chen_dry["RHO"]*1000).where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_dryweakinv_nic = (chen_dryweakinv["lag_NIC"]*chen_dryweakinv["RHO"]*1000).where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud ice-crystal number concentration in Chen_CONTROL between hours 2 and 8: {chen_nic:.4f} particles/L")
print(f"Average in-cloud ice-crystal number concentration in Chen_DRY between hours 2 and 8: {chen_dry_nic:.4f} particles/L")
print(f"Average in-cloud ice-crystal number concentration in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_nic:.4f} particles/L")
print(f"Average in-cloud ice-crystal number concentration in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_nic:.4f} particles/L")

print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_nic - chen_nic)/chen_nic:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(chen_dryweakinv_nic - chen_dry_nic)/chen_dry_nic:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_nic - chen_weakinv_nic)/chen_weakinv_nic:.3%}")

In [ ]:
chen_dry_dep = (chen_dry["lag_DEP"]).where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_dryweakinv_dep = (chen_dryweakinv["lag_DEP"]).where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average total in-cloud deposition rate in Chen_CONTROL between hours 2 and 8: {chen_dep:.3e} g/kg/s")
print(f"Average total in-cloud deposition rate in Chen_DRY between hours 2 and 8: {chen_dry_dep:.3e} g/kg/s")
print(f"Average total in-cloud deposition rate in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_dep:.3e} g/kg/s")
print(f"Average total in-cloud deposition rate in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_dep:.3e} g/kg/s")

print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_dep - chen_dep)/chen_dep:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(chen_dryweakinv_dep - chen_dry_dep)/chen_dry_dep:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_dep - chen_weakinv_dep)/chen_weakinv_dep:.3%}")


In [ ]:
cloudtop_dry = find_top(chen_dry["lag_QC"], 1e-2).values
topheights_dry = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_dry[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_dryweakinv = find_top(chen_dryweakinv["lag_QC"], 1e-2).values
topheights_dryweakinv = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_dryweakinv[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])

cldtoprad_control = chen_control['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_control['z'].sel(time = slice(1/12, 1/3))).mean()
cldtoprad_dry = chen_dry['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dry['z'].sel(time = slice(1/12, 1/3))).mean()
cldtoprad_weakinv = chen_weakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_weakinv['z'].sel(time = slice(1/12, 1/3))).mean()
cldtoprad_dryweakinv = chen_dryweakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dryweakinv['z'].sel(time = slice(1/12, 1/3))).mean()
print(f"Average Cloud-top longwave cooling rate in Chen_CONTROL between hours 2 and 8: {cldtoprad_control:.3f} K/day")
print(f"Average Cloud-top longwave cooling rate in Chen_DRY between hours 2 and 8: {cldtoprad_dry:.3f} K/day")
print(f"Average Cloud-top longwave cooling rate in Chen_WEAKINV between hours 2 and 8: {cldtoprad_weakinv:.3f} K/day")
print(f"Average Cloud-top longwave cooling rate in Chen_DRY_WEAKINV between hours 2 and 8: {cldtoprad_dryweakinv:.3f} K/day")

print(f"Relative difference between CONTROL and DRY_WEAKINV: {(cldtoprad_dryweakinv - cldtoprad_control)/cldtoprad_control:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(cldtoprad_dryweakinv - cldtoprad_dry)/cldtoprad_dry:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(cldtoprad_dryweakinv - cldtoprad_weakinv)/cldtoprad_weakinv:.3%}")

print(f"Cloud-top radiative cooling rates in DRY_WEAKINV at hour 2: {chen_dryweakinv['RADQRLW'].sel(time = 1/12, method = 'nearest').interp(z = topheights_dryweakinv['z'].sel(time = 1/12, method = 'nearest')).mean():.3f} K/day")
print(f"Cloud-top radiative cooling rates in DRY_WEAKINV at hour 8: {chen_dryweakinv['RADQRLW'].sel(time = 1/3, method = 'nearest').interp(z = topheights_dryweakinv['z'].sel(time = 1/3, method = 'nearest')).mean():.3f} K/day")

print(f"LWP in DRY_WEAKINV at hour 2: {(chen_dryweakinv['lag_QC']*chen_dryweakinv['RHO']).sel(time = 1/12, method = 'nearest').integrate(coord = 'z'):.3f} g/m^2")
print(f"LWP in DRY_WEAKINV at hour 8: {(chen_dryweakinv['lag_QC']*chen_dryweakinv['RHO']).sel(time = 1/3, method = 'nearest').integrate(coord = 'z'):.3f} g/m^2")

In [ ]:
cloudbase_dry = find_top(chen_dry["lag_QC"][::-1], 1e-2);
cloudbase_dryweakinv = find_top(chen_dryweakinv["lag_QC"][::-1], 1e-2)
cloudthick_dry = (cloudtop_dry - cloudbase_dry).sel(time = slice(1/12, 1/3)).mean()
cloudthick_dryweakinv = (cloudtop_dryweakinv - cloudbase_dryweakinv).sel(time = slice(1/12, 1/3)).mean()
print(f"Average cloud thickness in Chen_CONTROL between hours 2 and 8: {cloudthick_control:.2f} m")
print(f"Average cloud thickness in Chen_DRY between hours 2 and 8: {cloudthick_dry:.2f} m")
print(f"Average cloud thickness in Chen_WEAKINV between hours 2 and 8: {cloudthick_weakinv:.2f} m")
print(f"Average cloud thickness in Chen_DRY_WEAKINV between hours 2 and 8: {cloudthick_dryweakinv:.2f} m")
print(f"Relative difference between Chen_CONTROL and Chen_DRY_WEAKINV: {(cloudthick_dryweakinv - cloudthick_control)/cloudthick_control:.3%}")
print(f"Relative difference between Chen_DRY and Chen_DRY_WEAKINV: {(cloudthick_dryweakinv - cloudthick_dry)/cloudthick_dry:.3%}")
print(f"Relative difference between Chen_WEAKINV and Chen_DRY_WEAKINV: {(cloudthick_dryweakinv - cloudthick_weakinv)/cloudthick_weakinv:.3%}")

### Sub-cloud humidity comparison

In [ ]:
print(f"Mean depth of ice sub-saturated layer in CONTROL: {sub_startheight_chen.sel(time = slice(1/12, 1/3)).mean():.1f} m")
print(f"Mean depth of ice sub-saturated layer in CONTROL: {sub_startheight_chendry.sel(time = slice(1/12, 1/3)).mean():.1f} m")
print(f"Mean depth of ice sub-saturated layer in CONTROL: {sub_startheight_weakinv.sel(time = slice(1/12, 1/3)).mean():.1f} m")
print(f"Mean depth of ice sub-saturated layer in CONTROL: {sub_startheight_dryweakinv.sel(time = slice(1/12, 1/3)).mean():.1f} m")

print(f"Difference in relative humidity in ice-subsaturated layer between CONTROL and DRY_WEAKINV: {chen_dryweakinv['lag_iRH'].sel(z = slice(150,410), time = slice(1/12, 1/3)).mean() - chen_control['lag_iRH'].sel(z = slice(150, 410), time = slice(1/12, 1/3)).mean():.2%}%")
#

### Nucleation rates comparision

In [ ]:
mean_nucrate_dry = chen_dry["lag_NUC"].where(chen_dry["lag_NUC"]>0).sel(time = slice(1/12, 1/3)).mean()
mean_nucrate_dryweakinv = chen_dryweakinv["lag_NUC"].where(chen_dryweakinv["lag_NUC"]>0).sel(time = slice(1/12, 1/3)).mean()
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in CONTROL: {mean_nucrate_control:.3e} mg/kg/s")
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in DRY: {mean_nucrate_dry:.3e} mg/kg/s")
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in WEAKINV: {mean_nucrate_weakinv:.3e} mg/kg/s")
print(f"Average heterogeneous nucleation rate between hours 2 and 8 in DRY_WEAKINV: {mean_nucrate_dryweakinv:.3e} mg/kg/s")
print(f"Relative difference between DRY_WEAKINV and CONTROL: {(mean_nucrate_dryweakinv - mean_nucrate_control)/mean_nucrate_control:.3%}")
print(f"Relative difference between DRY_WEAKINV and DRY: {(mean_nucrate_dryweakinv - mean_nucrate_dry)/mean_nucrate_dry:.3%}")
print(f"Relative difference between DRY_WEAKINV and WEAKINV: {(mean_nucrate_dryweakinv - mean_nucrate_weakinv)/mean_nucrate_weakinv:.3%}")

vertint_nuc_dry = (chen_dry["lag_NUC"]*chen_dry["RHO"]).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
vertint_nuc_dryweakinv = (chen_dryweakinv["lag_NUC"]*chen_dryweakinv["RHO"]).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
print(f"Vertically integrated nucleation rate between hours 2 and 8 in CONTROL: {vertint_nuc_control:.3e} mg/m^2/s")
print(f"Vertically integrated nucleation rate between hours 2 and 8 in DRY: {vertint_nuc_dry:.3e} mg/m^2/s")
print(f"Vertically integrated nucleation rate between hours 2 and 8 in WEAKINV: {vertint_nuc_weakinv:.3e} mg/m^2/s")
print(f"Vertically integrated nucleation rate between hours 2 and 8 in DRY_WEAKINV: {vertint_nuc_dryweakinv:.3e} mg/m^2/s")
print(f"Relative difference between DRY_WEAKINV and CONTROL: {(vertint_nuc_dryweakinv - vertint_nuc_control)/vertint_nuc_control:.3%}")
print(f"Relative difference between DRY_WEAKINV and DRY: {(vertint_nuc_dryweakinv - vertint_nuc_dry)/vertint_nuc_dry:.3%}")
print(f"Relative difference between DRY_WEAKINV and WEAKINV: {(vertint_nuc_dryweakinv - vertint_nuc_weakinv)/vertint_nuc_weakinv:.3%}")

print(f"Relative difference in boundary layer height between DRY_WEAKINV and CONTROL: {(cloudtop_dryweakinv[240:960].mean() - cloudtop_control[240:960].mean())/cloudtop_control[240:960].mean():.3%}")
print(f"Relative difference in boundary layer height between DRY_WEAKINV and DRY: {(cloudtop_dryweakinv[240:960].mean() - cloudtop_dry[240:960].mean())/cloudtop_dry[240:960].mean():.3%}")
print(f"Relative difference in boundary layer height between DRY_WEAKINV and WEAKINV: {(cloudtop_dryweakinv[240:960].mean() - cloudtop_weakinv[240:960].mean())/cloudtop_weakinv[240:960].mean():.3%}")

### IN recycling

In [ ]:
subcldin_dry = (10**6*chen_dry["lag_NINC"]*chen_dry["RHO"]).sel(z = slice(0,700)).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()
subcldin_dryweakinv = (10**6*chen_dryweakinv["lag_NINC"]*chen_dryweakinv["RHO"]).sel(z = slice(0,700)).sel(time = slice(1/12, 1/3)).integrate(coord = "z").mean()

print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_CONTROL: {subcldin_control:.3e} particles/m^2")
print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_DRY: {subcldin_dry:.3e} particles/m^2")
print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_WEAKINV: {subcldin_weakinv:.3e} particles/m^2")
print(f"average vertically-integrated below-cloud IN between hours 2 and 8 in Chen_DRY_WEAKINV: {subcldin_dryweakinv:.3e} particles/m^2")

print(f"Relative difference between CONTROL and DRY_WEAKINV: {(subcldin_dryweakinv - subcldin_control)/subcldin_control:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(subcldin_dryweakinv - subcldin_dry)/subcldin_dry:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(subcldin_dryweakinv - subcldin_weakinv)/subcldin_weakinv:.3%}")

print(f"Relative difference in cloud base height between DRY_WEAKINV and CONTROL: {(cloudbase_dryweakinv[240:960].mean() - cloudbase_control[240:960].mean())/cloudbase_control[240:960].mean():.3%}")
print(f"Relative difference in cloud base height between DRY_WEAKInV and DRY: {(cloudbase_dryweakinv[240:960].mean() - cloudbase_dry[240:960].mean())/cloudbase_dry[240:960].mean():.3%}")
print(f"Relative difference in cloud base height between DRY_WEAKINV and WEAKINV: {(cloudbase_dryweakinv[240:960].mean() - cloudbase_weakinv[240:960].mean())/cloudbase_weakinv[240:960].mean():.3%}")

chen_weakinv_subfraction = chen_weakinv["lag_NIC"].sel(z=5, method = "nearest") / chen_weakinv["lag_NIC"].sel(z = sub_startheight_weakinv)
chen_dryweakinv_subfraction = chen_dryweakinv["lag_NIC"].sel(z=5, method = "nearest") / chen_dryweakinv["lag_NIC"].sel(z = sub_startheight_dryweakinv)
print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_CONTROL: {chen_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")
print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_DRY: {chen_dry_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")
print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_WEAKINV: {chen_weakinv_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")
print(f"Average fraction of ice crystals at the surface compared to the sublimation start altitude in Chen_DRY_WEAKINV: {chen_dryweakinv_subfraction.sel(time = slice(1/12, 1/3)).mean():.3%}")

### TKE Comparison

In [ ]:
chen_dry_tkepath = chen_dry["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
chen_dryweakinv_tkepath = chen_dryweakinv["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Mean TKE path Chen_CONTROL between hours 2 and 8: {chen_tkepath:.3f} g/kg/s")
print(f"Mean TKE path Chen_DRY between hours 2 and 8: {chen_dry_tkepath:.3f} g/kg/s")
print(f"Mean TKE path Chen_WEAKINV between hours 2 and 8: {chen_weakinv_tkepath:.3f} g/kg/s")
print(f"Mean TKE path Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_tkepath:.3f} g/kg/s")

print(f"Relative difference between CONTROL and DRY_WEAKINV: {(chen_dryweakinv_tkepath - chen_tkepath)/chen_tkepath:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(chen_dryweakinv_tkepath - chen_dry_tkepath)/chen_dry_tkepath:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(chen_dryweakinv_tkepath - chen_weakinv_tkepath)/chen_weakinv_tkepath:.3%}")


tkemean_dry = chen_dry["TKE"].where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
tkemean_dryweakinv = chen_dryweakinv["TKE"].where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Mean in-cloud TKE in Chen_CONTROL: {tkemean_control:.3f} m^2/s^2")
print(f"Mean in-cloud TKE in Chen_DRY: {tkemean_dry:.3f} m^2/s^2")
print(f"Mean in-cloud TKE in Chen_WEAKINV: {tkemean_weakinv:.3f} m^2/s^2")
print(f"Mean in-cloud TKE Chen_DRY_WEAKINV: {tkemean_dryweakinv:.3f} m^2/s^2")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(tkemean_dryweakinv - tkemean_control)/tkemean_control:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(tkemean_dryweakinv - tkemean_dry)/tkemean_dry:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(tkemean_dryweakinv - tkemean_weakinv)/tkemean_weakinv:.3%}")

w2mean_dry = chen_dry["W2"].where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
w2mean_dryweakinv = chen_dryweakinv["W2"].where(chen_dryweakinv["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Mean in-cloud vertical velocity variance in Chen_CONTROL: {w2mean_control:.3f} m^2/s^2")
print(f"Mean in-cloud vertical velocity variance in Chen_DRY: {w2mean_dry:.3f} m^2/s^2")
print(f"Mean in-cloud vertical velocity variance in Chen_WEAKINV: {w2mean_weakinv:.3f} m^2/s^2")
print(f"Mean in-cloud vertical velocity variance in Chen_DRY_WEAKINV: {w2mean_dryweakinv:.3f} m^2/s^2")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(w2mean_dryweakinv - w2mean_control)/w2mean_control:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(w2mean_dryweakinv - w2mean_dry)/w2mean_dry:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(w2mean_dryweakinv - w2mean_weakinv)/w2mean_weakinv:.3%}")

### Find Entrainment Metrics

In [ ]:
subv = -0.00412 #Large-scale subsidence velocity at cloud top, m/s
we_control = (np.gradient(cloudtop_control, 30) - subv)[240:960].mean()
we_dry = (np.gradient(cloudtop_dry, 30) - subv)[240:960].mean()
we_weakinv = (np.gradient(cloudtop_weakinv, 30) - subv)[240:960].mean()
we_dryweakinv = (np.gradient(cloudtop_dryweakinv, 30) - subv)[240:960].mean()
print(f"Average entrainment velocity in Chen_CONTROL between hours 2 and 8: {1000*we_control:.3f} mm/s")
print(f"Average entrainment velocity in Chen_DRY between hours 2 and 8: {1000*we_dry:.3f} mm/s")
print(f"Average entrainment velocity in Chen_WEAKINV between hours 2 and 8: {1000*we_weakinv:.3f} mm/s")
print(f"Averate entrainment velocity in Chen_DRYWEAKINV between hours 2 and 8: {1000*we_dryweakinv:.3f} mm/s")
print(f"Relative difference between CONTROL and WEAKINV: {(we_weakinv - we_control)/we_control:.3%}\n")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(we_dryweakinv - we_control)/we_control:.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(we_dryweakinv - we_dry)/we_dry:.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(we_dryweakinv - we_weakinv)/we_weakinv:.3%}")

rhent_control = (chen_control["RELH"].interp(z = cloudtop_control + 50).sel(time = slice(1/12, 1/3)).mean())
rhent_dry = (chen_dry["RELH"].interp(z = cloudtop_dry + 50).sel(time = slice(1/12, 1/3)).mean())
rhent_weakinv = (chen_weakinv["RELH"].interp(z = cloudtop_weakinv + 50).sel(time = slice(1/12, 1/3)).mean())
rhent_dryweakinv = (chen_dryweakinv["RELH"].interp(z = cloudtop_dryweakinv + 50).sel(time = slice(1/12, 1/3)).mean())
print(f"Relative humidity of air 20 meters above cloud top in Chen_CONTROL between hours 2 and 8: {rhent_control:.1f}%")
print(f"Relative humidity of air 20 meters above cloud top in Chen_DRY between hours 2 and 8: {rhent_dry:.1f}%")
print(f"Relative humidity of air 20 meters above cloud top in Chen_WEAKINV between hours 2 and 8: {rhent_weakinv:.1f}%")
print(f"Relative humidity of air 20 meters above cloud top in Chen_DRY_WEAKINV between hours 2 and 8: {rhent_dryweakinv:.1f}%")

rhbl_control = (chen_control["RELH"].sel(z = slice(150,400)).sel(time = slice(1/12, 1/3)).mean())
rhbl_dry = (chen_dry["RELH"].sel(z = slice(150,400)).sel(time = slice(1/12, 1/3)).mean())
rhbl_weakinv = (chen_weakinv["RELH"].sel(z = slice(150,400)).sel(time = slice(1/12, 1/3)).mean())
rhbl_dryweakinv = (chen_dryweakinv["RELH"].sel(z = slice(150,400)).sel(time = slice(1/12, 1/3)).mean())
print(f"Difference in relative humidity w.r.t. ice of the sub-cloud boundary layer between Chen_DRY and Chen_CONTROL between hours 2 and 8: {rhbl_dry - rhbl_control:.2f}%")
print(f"Difference in relative humidity w.r.t. ice of the sub-cloud boundary layer between Chen_WEAKINV and Chen_CONTROL between hours 2 and 8: {rhbl_weakinv - rhbl_control:.2f}%")
print(f"Difference in relative humidity w.r.t. ice of the sub-cloud boundary layer between Chen_DRY_WEAKINV and Chen_CONTROL between hours 2 and 8: {rhbl_dryweakinv - rhbl_control:.2f}%")
#The lowest 150m are excluded because of differences in mixing between simulations meaning that there's a greater reservoir of inaccessible near-surface moisture in WEAKINV than CONTROL

In [ ]:
print(f"Average cloud base lowering rate between hours 2 and 8 in Chen_CONTROL: {-1000*np.gradient(cloudbase_control, 30)[240:960].mean():.3f} mm/s")
print(f"Average cloud base lowering rate between hours 2 and 8 in Chen_DRYL: {-1000*np.gradient(cloudbase_dry, 30)[240:960].mean():.3f} mm/s")
print(f"Average cloud base lowering rate between hours 2 and 8 in Chen_WEAKINV: {-1000*np.gradient(cloudbase_weakinv, 30)[240:960].mean():.3f} mm/s")
print(f"Average cloud base lowering rate between hours 2 and 8 in Chen_DRY_WEAKINV: {-1000*np.gradient(cloudbase_dryweakinv, 30)[240:960].mean():.3f} mm/s")
print(f"Relative difference between CONTROL and WEAKINV: {(np.gradient(cloudbase_weakinv, 30)[240:960].mean() - np.gradient(cloudbase_control, 30)[240:960].mean())/np.gradient(cloudbase_control, 30)[240:960].mean():.3%}\n")
print(f"Relative difference between CONTROL and DRY_WEAKINV: {(np.gradient(cloudbase_dryweakinv, 30)[240:960].mean() - np.gradient(cloudbase_control, 30)[240:960].mean())/np.gradient(cloudbase_control, 30)[240:960].mean():.3%}")
print(f"Relative difference between DRY and DRY_WEAKINV: {(np.gradient(cloudbase_dryweakinv, 30)[240:960].mean() - np.gradient(cloudbase_dry, 30)[240:960].mean())/np.gradient(cloudbase_dry, 30)[240:960].mean():.3%}")
print(f"Relative difference between WEAKINV and DRY_WEAKINV: {(np.gradient(cloudbase_dryweakinv, 30)[240:960].mean() - np.gradient(cloudbase_weakinv, 30)[240:960].mean())/np.gradient(cloudbase_weakinv, 30)[240:960].mean():.3%}")

## Comparisons between INFLUX, DRY, and DRY_INFLUX

In [ ]:
dryinfluxpath = input("Enter the path to the time-height statistics file for the DRY_INFLUX simulation: ").strip()
chen_dryinflux = xr.open_dataset(dryinfluxpath).transpose()

In [ ]:
chen_dryinflux_lwp = (chen_dryinflux["lag_QC"]*chen_dryinflux["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
chen_dry_lwp = (chen_dry["lag_QC"]*chen_dry["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average LWP in CHEN_CONTROL between hours 2 and 8: {chen_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_DRY between hours 2 and 8: {chen_dry_lwp:3f} g/m^2")
print(f"Average LWP in CHEN_WEAKINV between hours 2 and 8: {chen_weakinv_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_INFLUX between hours 2 and 8: {chen_influx_lwp:.3f} g/m^2")
print(f"Average LWP in CHEN_DRY_INFLUX between hours 2 and 8: {chen_dryinflux_lwp:.3f} g/m^2")
print(f"Relative difference between DRY_INFLUX and INFLUX: {(chen_dryinflux_lwp - chen_influx_lwp)/chen_influx_lwp:.3%}")
print(f"Relative difference between DRY_INFLUX and DRY: {(chen_dryinflux_lwp - chen_dry_lwp)/chen_dry_lwp:.3%}")
print(f"Relative difference between DRY_INFLUX and CONTROL: {(chen_dryinflux_lwp - chen_lwp)/chen_lwp:.3%}")

In [ ]:
chen_dryinflux_iwp = (chen_dryinflux["lag_QI"]*chen_dryinflux["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
chen_dry_iwp = (chen_dry["lag_QI"]*chen_dry["RHO"]).integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Average IWP in CHEN_CONTROL between hours 2 and 8: {chen_iwp:.3f} g/m^2")
print(f"Average IWP in CHEN_DRY between hours 2 and 8: {chen_dry_iwp:3f} g/m^2")
print(f"Average IWP in CHEN_WEAKINV between hours 2 and 8: {chen_weakinv_iwp:.3f} g/m^2")
print(f"Average IWP in CHEN_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_iwp:.3f} g/m^2")
print(f"Average IWP in CHEN_INFLUX between hours 2 and 8: {chen_influx_iwp:.3f} g/m^2")
print(f"Average IWP in CHEN_DRY_INFLUX between hours 2 and 8: {chen_dryinflux_iwp:.3f} g/m^2")
print(f"Relative difference between DRY_INFLUX and INFLUX: {(chen_dryinflux_iwp - chen_influx_iwp)/chen_influx_iwp:.3%}")
print(f"Relative difference between DRY_INFLUX and DRY: {(chen_dryinflux_iwp - chen_dry_iwp)/chen_dry_iwp:.3%}")
print(f"Relative difference between DRY_INFLUX and CONTROL: {(chen_dryinflux_iwp - chen_iwp)/chen_iwp:.3%}")

### Plot time series of in-cloud condensation and deposition rates

In [ ]:
from matplotlib.ticker import ScalarFormatter
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], 0, 0.06, 0.94, 0.06)
    ax.set_xlabel("Elapsed Time (h)")
    ax.set_xlim(0,8)
ax1.set_ylabel("In-Cloud Mean Deposition Rate ($g \ kg^{-1} \ s^{-1}$)")
ax2.set_ylabel("In-Cloud Mean Condensation Rate ($g \ kg^{-1} \ s^{-1}$)")
ax2.set_ylim(7.5e-5, 2e-4)
ax1.set_ylim(0, 2e-5)
ax1.axvline(1, color = "black", linestyle = ":")
ax1.plot(24*chen_control["time"], chen_control["lag_DEP"].where(chen_control["lag_QC"]>1e-2).mean(dim = "z"), color = "Blue", label = "CONTROL")
ax1.plot(24*chen_control["time"], chen_dry["lag_DEP"].where(chen_dry["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange", label = "DRY", linestyle = "-")
ax1.plot(24*chen_control["time"], chen_influx["lag_DEP"].where(chen_influx["lag_QC"]>1e-2).mean(dim = "z"), color = "Blue", label = "INFLUX", linestyle = "--")
ax1.plot(24*chen_control["time"], chen_dryinflux["lag_DEP"].where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange", label = "CDRY_INFLUX", linestyle = "--")
ax1.plot(24*chen_control["time"], chen_weakinv["lag_DEP"].where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_control["time"], chen_dryweakinv["lag_DEP"].where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "darkcyan", label = "DRY_WEAKINV")
ax2.plot(24*chen_control["time"], chen_control["lag_CND"].where(chen_control["lag_QC"]>1e-2).mean(dim = "z"), color = "Blue")
ax2.plot(24*chen_control["time"], chen_dry["lag_CND"].where(chen_dry["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange", linestyle = "-")
ax2.plot(24*chen_control["time"], chen_influx["lag_CND"].where(chen_influx["lag_QC"]>1e-2).mean(dim = "z"), color = "Blue", linestyle = "--")
ax2.plot(24*chen_control["time"], chen_dryinflux["lag_CND"].where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange", linestyle = "--")
ax2.plot(24*chen_control["time"], chen_weakinv["lag_CND"].where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "turquoise")
ax2.plot(24*chen_control["time"], chen_dryweakinv["lag_CND"].where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "darkcyan")
fig.legend(ncols = 3, loc = "outside lower center")
fig.savefig("paperfigs/allsims_dep_cond_comp.png")
plt.close()

### Use Gaussian smoothing to make it a bit more readable

In [ ]:
from matplotlib.ticker import ScalarFormatter
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], 0, 0.06, 0.94, 0.06)
    ax.set_xlabel("Elapsed Time (h)")
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
ax1.set_ylabel("In-Cloud Mean Deposition Rate ($g \ kg^{-1} \ s^{-1}$)")
ax2.set_ylabel("In-Cloud Mean Condensation Rate ($g \ kg^{-1} \ s^{-1}$)")
ax2.set_ylim(7.5e-5, 2e-4)
ax1.set_ylim(0, 2e-5)
ax1.plot(24*chen_control["time"], np.concatenate((chen_control["lag_DEP"].where(chen_control["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_control["lag_DEP"].where(chen_control["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "Blue", label = "CONTROL")
ax1.plot(24*chen_control["time"], np.concatenate((chen_dry["lag_DEP"].where(chen_dry["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_dry["lag_DEP"].where(chen_dry["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "darkorange", label = "DRY", linestyle = "-")
ax1.plot(24*chen_control["time"], np.concatenate((chen_influx["lag_DEP"].where(chen_influx["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_influx["lag_DEP"].where(chen_influx["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "Blue", label = "INFLUX", linestyle = "--")
ax1.plot(24*chen_control["time"], np.concatenate((chen_dryinflux["lag_DEP"].where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_dryinflux["lag_DEP"].where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "darkorange", label = "DRY_INFLUX", linestyle = "--")
ax1.plot(24*chen_control["time"], np.concatenate((chen_weakinv["lag_DEP"].where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_weakinv["lag_DEP"].where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_control["time"], np.concatenate((chen_dryweakinv["lag_DEP"].where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(0, 1/24)).values, gaussian_filter(chen_dryweakinv["lag_DEP"].where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z").sel(time = slice(1/24, 1/3)).values, (10), mode = "nearest"))), color = "darkcyan", label = "DRY_WEAKINV")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_control["lag_CND"].where(chen_control["lag_QC"]>1e-2).mean(dim = "z").values, (10), mode = "nearest"), color = "Blue")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_dry["lag_CND"].where(chen_dry["lag_QC"]>1e-2).mean(dim = "z").values, (10), mode = "nearest"), color = "darkorange", linestyle = "-")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_influx["lag_CND"].where(chen_influx["lag_QC"]>1e-2).mean(dim = "z"), (10), mode = "nearest"), color = "Blue", linestyle = "--")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_dryinflux["lag_CND"].where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z"), (10), mode = "nearest"), color = "darkorange", linestyle = "--")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_weakinv["lag_CND"].where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z"), (10), mode = "nearest"), color = "turquoise")
ax2.plot(24*chen_control["time"], gaussian_filter(chen_dryweakinv["lag_CND"].where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z"), (10), mode = "nearest"), color = "darkcyan")
fig.legend(ncols = 3, loc = "outside lower center")
fig.savefig("paperfigs/allsims_dep_cond_comp_filtered.png")
plt.close()

In [ ]:
fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2, 2, figsize = (6, 4), dpi = 250)
axlabels = ["(a)", "(b)", "(c)", "(d)"]
l,w,b,h = 0, 0.06, 0.9, 0.1
for i, ax in enumerate((ax11, ax12, ax21, ax22)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.axvline(1, color = "black", linestyle = ":")
    ax.set_ylim(400, 1000)
    ax.set_xlim(0,8)
ax11.set_ylabel("Altitude AMSL (m)")
ax21.set_ylabel("Altitude AMSL (m)")
ax21.set_xlabel("Elapsed Time (h)")
ax22.set_xlabel("Elapsed Time (h)")
ax11.set_title("Chen_CONTROL")
ax12.set_title("Chen_INFLUX")
ax21.set_title("Chen_DRY")
ax22.set_title("Chen_DRY_INFLUX")
condmp = ax11.pcolormesh(24*chen_control["time"], chen_control["z"], chen_control["lag_CND"], vmin = 0, vmax = 2e-4, cmap = "Greens")
ax11.contour(24*chen_control["time"], chen_control["z"], chen_control["lag_QC"], levels = [1e-2], colors = "black")
ax12.pcolormesh(24*chen_control["time"], chen_control["z"], chen_influx["lag_CND"], vmin = 0, vmax = 2e-4, cmap = "Greens")
ax12.contour(24*chen_control["time"], chen_control["z"], chen_influx["lag_QC"], levels = [1e-2], colors = "black")
ax21.pcolormesh(24*chen_control["time"], chen_control["z"], chen_dry["lag_CND"], vmin = 0, vmax = 2e-4, cmap = "Greens")
ax21.contour(24*chen_control["time"], chen_control["z"], chen_dry["lag_QC"], levels = [1e-2], colors = "black")
ax22.pcolormesh(24*chen_control["time"], chen_control["z"], chen_dryinflux["lag_CND"], vmin = 0, vmax = 2e-4, cmap = "Greens")
ax22.contour(24*chen_control["time"], chen_control["z"], chen_dryinflux["lag_QC"], levels = [1e-2], colors = "black")
fig.colorbar(condmp, ax = [ax11, ax12, ax21, ax22], orientation = "horizontal", aspect = 35, fraction = 0.05, extend = "max", label = "Condensation Rate ($g \ kg^{-1} \ s^{-1}$)")
fig.savefig("paperfigs/supplemental_figure5_condensationcomp.png")
plt.close()

In [ ]:
cloudtop_influx = find_top(chen_influx["lag_QC"], 1e-2).values
topheights_influx = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_influx[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_dry = find_top(chen_dry["lag_QC"], 1e-2).values
topheights_dry = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_dry[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_dryinflux = find_top(chen_dryinflux["lag_QC"], 1e-2).values
topheights_dryinflux = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_dryinflux[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_control = find_top(chen_control["lag_QC"], 1e-2).values
topheights_control = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_control[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_weakinv = find_top(chen_weakinv["lag_QC"], 1e-2).values
topheights_weakinv = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_weakinv[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])
cloudtop_dryweakinv = find_top(chen_dryweakinv["lag_QC"], 1e-2).values
topheights_dryweakinv = xr.DataArray(coords = {"z": (("relheight", "time"), cloudtop_dryweakinv[None,:]+np.linspace(-40, 0, 41)[:,None]), "relheight": np.linspace(-40, 0, 41), "time": chen_control["time"]},
                                 dims = ["relheight", "time"])

### Comparisons of longwave cooling and TKE

In [ ]:
print(f"Average cloud-top radiative cooling in Chen_CONTROL between hours 2 and 8: {chen_control['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_control['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_DRY between hours 2 and 8: {chen_dry['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dry['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_WEAKINV between hours 2 and 8: {chen_weakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_weakinv['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dryweakinv['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_INFLUX between hours 2 and 8: {chen_influx['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_influx['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")
print(f"Average cloud-top radiative cooling in Chen_DRY_INFLUX between hours 2 and 8: {chen_dryinflux['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dryinflux['z'].sel(time = slice(1/12, 1/3))).mean():.3f} K/day")

meanrad_influx = chen_influx['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_influx['z'].sel(time = slice(1/12, 1/3))).mean()
meanrad_dry = chen_dry['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dry['z'].sel(time = slice(1/12, 1/3))).mean()
meanrad_dryinflux = chen_dryinflux['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dryinflux['z'].sel(time = slice(1/12, 1/3))).mean()
meanrad_control = chen_control['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_control['z'].sel(time = slice(1/12, 1/3))).mean()
meanrad_weakinv = chen_weakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_weakinv['z'].sel(time = slice(1/12, 1/3))).mean()
meanrad_dryweakinv = chen_dryweakinv['RADQRLW'].sel(time = slice(1/12, 1/3)).interp(z = topheights_dryweakinv['z'].sel(time = slice(1/12, 1/3))).mean()

print(f"Relative difference between INFLUX and CONTROL: {((meanrad_influx - meanrad_control)/meanrad_control):.3%}")
print(f"Relative difference between DRY and CONTROL: {((meanrad_dry - meanrad_control)/meanrad_control):.3%}")
print(f"Relative difference between WEAKINV and CONTROL: {((meanrad_weakinv - meanrad_control)/meanrad_control):.3%}")
print(f"Relative difference between DRY_INFLUX and CONTROL: {((meanrad_dryinflux - meanrad_control)/meanrad_control):.3%}")
print(f"Relative difference between DRY_INFLUX and INFLUX: {((meanrad_dryinflux - meanrad_influx)/meanrad_dry):.3%}")
print(f"Relative difference between DRY_INFLUX and DRY: {((meanrad_dryinflux - meanrad_dry)/meanrad_dry):.3%}")

In [ ]:
chen_dryinflux_tkepath = chen_dryinflux["TKE"].integrate(coord = "z").sel(time = slice(1/12, 1/3)).mean()
print(f"Mean TKE path in Chen_CONTROL between hours 2 and 8: {chen_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_DRY between hours 2 and 8: {chen_dry_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_INFLUX between hours 2 and 8: {chen_influx_tkepath:.3f} m^3 s^-2")
print(f"Mean TKE path in Chen_DRY_INFLUX between hours 2 and 8: {chen_dryinflux_tkepath:.3f} m^3 s^-2")
print(f"Relative Difference between DRY_INFLUX and CONTROL: {((chen_dryinflux_tkepath - chen_tkepath)/chen_tkepath):.3%}")
print(f"Relative Difference between DRY_INFLUX and INFLUX: {((chen_dryinflux_tkepath - chen_influx_tkepath)/chen_influx_tkepath):.3%}")
print(f"Relative Difference between DRY_INFLUX and DRY: {((chen_dryinflux_tkepath - chen_dry_tkepath)/chen_dry_tkepath):.3%}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
print(ax1)
print(ax2)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_xlim(0,8)
    ax.set_xlabel("Elapsed Time (h)")
    ax.axvline(1, color = "black", linestyle = ":")
ax1.set_ylabel("Cloud-Top Radiative Cooling ($K \ day^{-1}$)")
ax2.set_ylabel("Vertically Integrated TKE ($m^{3} \ s^{-2}$)")
ax1.set_ylim(0, 90)
ax2.set_ylim(0, 225)
ax1.plot(24*chen_control["time"], -1*chen_control["RADQRLW"].interp(z = topheights_control['z']).mean(dim = "relheight"), color = "blue", label = "CONTROL")
ax1.plot(24*chen_control["time"], -1*chen_dry["RADQRLW"].interp(z = topheights_dry["z"]).mean(dim = "relheight"), color = "darkorange", label = "DRY")
ax1.plot(24*chen_control["time"], -1*chen_influx["RADQRLW"].interp(z = topheights_influx["z"]).mean(dim = "relheight"), color = "blue", linestyle = "--", label = "INFLUX")
ax1.plot(24*chen_control["time"], -1*chen_dryinflux["RADQRLW"].interp(z = topheights_dryinflux["z"]).mean(dim = "relheight"), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax1.plot(24*chen_control["time"], -1*chen_weakinv["RADQRLW"].interp(z = topheights_weakinv["z"]).mean(dim = "relheight"), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_control["time"], -1*chen_dryweakinv["RADQRLW"].interp(z = topheights_dryweakinv["z"]).mean(dim = "relheight"), color = "darkcyan", label = "DRY_WEAKINV")

ax2.plot(24*chen_control["time"], chen_control["TKE"].integrate(coord = "z"), color = "blue")
ax2.plot(24*chen_control["time"], chen_dry["TKE"].integrate(coord = "z"), color = "darkorange")
ax2.plot(24*chen_control["time"], chen_influx["TKE"].integrate(coord = "z"), color = "blue", linestyle = "--")
ax2.plot(24*chen_control["time"], chen_dryinflux["TKE"].integrate(coord = "z"), color = "darkorange", linestyle = "--")
ax2.plot(24*chen_control["time"], chen_weakinv["TKE"].integrate(coord = "z"), color = "turquoise")
ax2.plot(24*chen_control["time"], chen_dryweakinv["TKE"].integrate(coord = "z"), color = "darkcyan")
fig.legend(loc = "outside lower center", ncols = 3)
fig.savefig("paperfigs/allsims_longwave_tkecomp.png")
plt.close()

### Deposition Rates per Particle

In [ ]:
chen_dry_depper = (chen_dry["lag_DEP"]/chen_dry["lag_NIC"]).where(chen_dry["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
chen_dryinflux_depper = (chen_dryinflux["lag_DEP"]/chen_dryinflux["lag_NIC"]).where(chen_dryinflux["lag_QC"]>1e-2).sel(time = slice(1/12, 1/3)).mean()
print(f"Average in-cloud deposition rate per particle in Chen_CONTROL between hours 2 and 8: {chen_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_DRY between hours 2 and 8: {chen_dry_depper:.4f} ug/particle/s")
print(f"Averate in-cloud deposition rate per particle in Chen_WEAKINV between hours 2 and 8: {chen_weakinv_depper:.4f} ug/particle/s")
print(f"Averate in-cloud deposition rate per particle in Chen_DRY_WEAKINV between hours 2 and 8: {chen_dryweakinv_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_INFLUX between hours 2 and 8: {chen_influx_depper:.4f} ug/particle/s")
print(f"Average in-cloud deposition rate per particle in Chen_DRY_INFLUX between hours 2 and 8: {chen_dryinflux_depper:.4f} ug/particle/s")
print(f"Relative difference between DRY_INFLUX and CONTROL: {(chen_dryinflux_depper - chen_depper)/chen_depper:.3%}")
print(f"Relative difference between DRY_INFLUX and DRY: {(chen_dryinflux_depper - chen_dry_depper)/chen_dry_depper:.3%}")
print(f"Relative difference between DRY_INFLUX and INFLUX: {(chen_dryinflux_depper - chen_influx_depper)/chen_influx_depper:.3%}")

### Particle radii below cloud

In [ ]:
sub_startheight_cheninflux = chen_control["z"].where(chen_influx["lag_iRH"]>1).min(dim = "z")
sub_startheight_chendryinflux = chen_control["z"].where(chen_dryinflux["lag_iRH"]>1).min(dim = "z")

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (3,3), dpi = 250)
ax1.set_ylabel("Ice Crystal Mean Radius ($\mu m$)")
ax1.set_xlabel("Elapsed Time (h)")
ax1.set_title("Time Series of Ice Crystal Mean Radius at \n Lowest Ice-Supersaturated Altitude")
ax1.axvline(1, color = "black", linestyle = ":")
ax1.set_xlim(0,8)
ax1.set_ylim(0,1200)
ax1.plot(24*chen_control["time"], chen_control["lag_iR"].sel(z=sub_startheight_chen), color = "blue", label = "CONTROL")
ax1.plot(24*chen_dry["time"], chen_dry["lag_iR"].sel(z=sub_startheight_chendry), color = "darkorange", label = "DRY")
ax1.plot(24*chen_control["time"], chen_influx["lag_iR"].sel(z = sub_startheight_cheninflux), color = "blue", linestyle = "--", label = "INFLUX")
ax1.plot(24*chen_control["time"], chen_dryinflux["lag_iR"].sel(z = sub_startheight_chendryinflux), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax1.plot(24*chen_control["time"], chen_weakinv["lag_iR"].sel(z=sub_startheight_weakinv), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_dry["time"], chen_dryweakinv["lag_iR"].sel(z=sub_startheight_dryweakinv), color = "darkcyan", label = "DRY_WEAKINV")
ax1.legend(loc = "lower right")
fig.savefig("paperfigs/supplemental_figure3_allsims_iceradii_substart.png")
plt.close()

In [ ]:
print(f"Average ice radius at sublimation start altitude in Chen_CONTROL: {chen_control['lag_iR'].sel(z=sub_startheight_chen).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")
print(f"Average ice radius at sublimation start altitude in Chen_DRY: {chen_dry['lag_iR'].sel(z=sub_startheight_chendry).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")
print(f"Average ice radius at sublimation start altitude in Chen_INFLUX: {chen_influx['lag_iR'].sel(z=sub_startheight_cheninflux).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")
print(f"Average ice radius at sublimation start altitude in Chen_DRY_INFLUX: {chen_dryinflux['lag_iR'].sel(z=sub_startheight_chendryinflux).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")
print(f"Average ice radius at sublimation start altitude in Chen_WEAKINV: {chen_weakinv['lag_iR'].sel(z=sub_startheight_weakinv).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")
print(f"Average ice radius at sublimation start altitude in Chen_DRY_WEAKINV: {chen_dryweakinv['lag_iR'].sel(z=sub_startheight_dryweakinv).sel(time = slice(1/12, 1/3)).mean():.3f} micrometers")

In [ ]:
print(f"Relative difference in radius between CONTROL and WEAKINV: {((chen_weakinv['lag_iR'].sel(z=sub_startheight_weakinv).sel(time = slice(1/12, 1/3)).mean() - chen_control['lag_iR'].sel(z=sub_startheight_chen).sel(time = slice(1/12, 1/3)).mean())/chen_control['lag_iR'].sel(z=sub_startheight_chen).sel(time = slice(1/12, 1/3)).mean()):.3%}") 

### Plot Precipitation Flux

In [ ]:
print(chen_control["lag_pQLF"])

In [ ]:
dt = 86400*(chen_control["time"][1]-chen_control["time"][0])
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i, ax in enumerate(fig.get_axes()):
   ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
ax2.axvline(1, color = "black", linestyle = ":")
ax2.set_ylim(0, 0.06)
ax1.set_xlim(40, 120)
ax1.set_ylim(0, 1400)
ax2.set_xlim(0,8)
ax2.plot(24*chen_control["time"], dt/(2.5*10**6)*chen_control["lag_pQLF"].isel(z = 1).cumsum(dim = "time"), color = "blue", label = "CONTROL")
ax2.plot(24*chen_influx["time"], dt/(2.5*10**6)*chen_influx["lag_pQLF"].isel(z=1).cumsum(dim = "time"), color = "blue", linestyle = "--", label = "INFLUX")
ax2.plot(24*chen_dry["time"], dt/(2.5*10**6)*chen_dry["lag_pQLF"].isel(z=1).cumsum(dim = "time"), color = "darkorange", label = "DRY")
ax2.plot(24*chen_dryinflux["time"], dt/(2.5*10**6)*chen_dryinflux["lag_pQLF"].isel(z=1).cumsum(dim = "time"), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax2.plot(24*chen_control["time"], dt/(2.5*10**6)*chen_weakinv["lag_pQLF"].isel(z=1).cumsum(dim = "time"), color = "turquoise", label = "WEAKINV")
ax2.plot(24*chen_control["time"], dt/(2.5*10**6)*chen_dryweakinv["lag_pQLF"].isel(z=1).cumsum(dim = "time"), color = "darkcyan", label = "DRY_WEAKINV")
ax1.plot(100*chen_control["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "blue")
ax1.plot(100*chen_influx["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "blue", linestyle = "--")
ax1.plot(100*chen_dry["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "darkorange")
ax1.plot(100*chen_dryinflux["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "darkorange", linestyle = "--")
ax1.plot(100*chen_weakinv["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "turquoise")
ax1.plot(100*chen_dryweakinv["lag_iRH"].sel(time = slice(1/12, 1/3)).mean(dim = "time"), chen_control["z"], color = "darkcyan")
ax2.set_ylabel("Total Accumulated Surface Precipitation (mm)")
ax2.set_xlabel("Elapsed Time (h)")
ax1.set_ylabel("Altitude (m AMSL)")
ax1.set_xlabel("Mean Relative Humidity Over Ice (%)")
fig.legend(loc = "outside lower center", ncols = 2)
fig.savefig("paperfigs/allsims_precip_rhice.png")
plt.close()

## Six-simulation line plots

### Plot LWP and IWP for all simulations

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i,ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
ax1.set_ylim(0, 40)
ax2.set_ylim(0, 10)
ax1.plot(24*chen_control["time"], (chen_control["lag_QC"]*chen_control["RHO"]).integrate(coord = "z"), color = "blue", label = "CONTROL")
ax1.plot(24*chen_control["time"], (chen_dry["lag_QC"]*chen_dry["RHO"]).integrate(coord = "z"), color = "darkorange", linestyle = "-", label = "DRY")
ax1.plot(24*chen_control["time"], (chen_influx["lag_QC"]*chen_influx["RHO"]).integrate(coord = "z"), color = "blue", linestyle = "--", label = "INFLUX",)
ax1.plot(24*chen_control["time"], (chen_dryinflux["lag_QC"]*chen_dryinflux["RHO"]).integrate(coord = "z"), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax1.plot(24*chen_control["time"], (chen_weakinv["lag_QC"]*chen_weakinv["RHO"]).integrate(coord = "z"), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_control["time"], (chen_dryweakinv["lag_QC"]*chen_dryweakinv["RHO"]).integrate(coord = "z"), color = "darkcyan", label = "DRY_WEAKINV")
ax2.plot(24*chen_control["time"], (chen_control["lag_QI"]*chen_control["RHO"]).integrate(coord = "z"), color = "blue")
ax2.plot(24*chen_control["time"], (chen_dry["lag_QI"]*chen_dry["RHO"]).integrate(coord = "z"), color = "darkorange")
ax2.plot(24*chen_control["time"], (chen_influx["lag_QI"]*chen_influx["RHO"]).integrate(coord = "z"), color = "blue", linestyle = "--")
ax2.plot(24*chen_control["time"], (chen_dryinflux["lag_QI"]*chen_dryinflux["RHO"]).integrate(coord = "z"), color = "darkorange", linestyle = "--")
ax2.plot(24*chen_control["time"], (chen_weakinv["lag_QI"]*chen_weakinv["RHO"]).integrate(coord = "z"), color = "turquoise")
ax2.plot(24*chen_control["time"], (chen_dryweakinv["lag_QI"]*chen_dryweakinv["RHO"]).integrate(coord = "z"), color = "darkcyan")
ax1.set_ylabel("Liquid Water Path ($g \ m^{-2}$)")
ax2.set_ylabel("Ice Water Path ($g \ m^{-2}$)")
ax1.set_xlabel("Elapsed Time (h)")
ax2.set_xlabel("Elapsed Time (h)")
fig.legend(ncols = 3, loc  = "outside lower center")
fig.savefig("paperfigs/allsims_lwp_iwp_timeseries.png")
plt.close()

### Plot ICNC and Vertically Integrated IN

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i,ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
# ax1.set_ylim(0, 1e6)
ax1.set_ylim(0, 1.25)
ax1.plot(24*chen_control["time"], 10**3*(chen_control["lag_NIC"]*chen_control["RHO"]).where(chen_control["lag_QC"]>1e-2).mean(dim = "z"), color = "blue")
ax1.plot(24*chen_control["time"], 10**3*(chen_dry["lag_NIC"]*chen_dry["RHO"]).where(chen_dry["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange")
ax1.plot(24*chen_control["time"], 10**3*(chen_influx["lag_NIC"]*chen_influx["RHO"]).where(chen_influx["lag_QC"]>1e-2).mean(dim = "z"), color = "blue", linestyle = "--")
ax1.plot(24*chen_control["time"], 10**3*(chen_dryinflux["lag_NIC"]*chen_dryinflux["RHO"]).where(chen_dryinflux["lag_QC"]>1e-2).mean(dim = "z"), color = "darkorange", linestyle = "--")
ax1.plot(24*chen_control["time"], 10**3*(chen_weakinv["lag_NIC"]*chen_weakinv["RHO"]).where(chen_weakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "turquoise")
ax1.plot(24*chen_control["time"], 10**3*(chen_dryweakinv["lag_NIC"]*chen_dryweakinv["RHO"]).where(chen_dryweakinv["lag_QC"]>1e-2).mean(dim = "z"), color = "darkcyan")
ax2.plot(24*chen_control["time"], 10**6*(chen_control["lag_NINC"]*chen_control["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "blue", label = "CONTROL")
ax2.plot(24*chen_control["time"], 10**6*(chen_dry["lag_NINC"]*chen_dry["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "darkorange", label = "DRY")
ax2.plot(24*chen_control["time"], 10**6*(chen_influx["lag_NINC"]*chen_influx["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "blue", linestyle = "--", label = "INFLUX")
ax2.plot(24*chen_control["time"], 10**6*(chen_dryinflux["lag_NINC"]*chen_dryinflux["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax2.plot(24*chen_control["time"], 10**6*(chen_weakinv["lag_NINC"]*chen_weakinv["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "turquoise", label = "WEAKINV")
ax2.plot(24*chen_control["time"], 10**6*(chen_dryweakinv["lag_NINC"]*chen_dryweakinv["RHO"]).sel(z = slice(0, 800)).integrate(coord = "z"), color = "darkcyan", label = "DRY_WEAKINV")
ax1.set_ylabel("Ice Crystal Number Concentration ($\# \ L^{-1}$)")
ax2.set_ylabel("Vertically Integrated IN ($\# \ m^{-2}$)")
ax2.set_xlabel("Elapsed Time (h)")
ax1.set_xlabel("Elapsed Time (h)")
fig.legend(ncols = 3, loc  = "outside lower center")
fig.savefig("paperfigs/allsims_icnc_invertint.png")
plt.close()

In [ ]:
print(f"Deposition-weighted in-cloud mean temperature in WEAKINV: {((chen_weakinv['TABS']*chen_weakinv['lag_DEP']).where(chen_weakinv['lag_QC']>1e-2)/(chen_weakinv['lag_DEP'].where(chen_weakinv['lag_QC']>1e-2))).sel(time = slice(1/12, 1/3)).mean():.2f} K")
print(f"Deposition-weighted in-cloud mean temperature in CONTROL: {((chen_control['TABS']*chen_control['lag_DEP']).where(chen_control['lag_QC']>1e-2)/(chen_control['lag_DEP'].where(chen_control['lag_QC']>1e-2))).sel(time = slice(1/12, 1/3)).mean():.2f} K")

In [ ]:
print(f"Ice mean lifetime just above cloud base in Chen_CONTROL: {chen_control['lag_ITMN'].sel(z=chen_control['z'].where(chen_control['lag_QC']>1e-2).min()+30).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime just above cloud base in Chen_DRY: {chen_dry['lag_ITMN'].sel(z=chen_dry['z'].where(chen_dry['lag_QC']>1e-2).min()+30).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime just above cloud base in Chen_INFLUX: {chen_influx['lag_ITMN'].sel(z=chen_influx['z'].where(chen_influx['lag_QC']>1e-2).min()+30).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime just above cloud base in Chen_DRY_INFLUX: {chen_dryinflux['lag_ITMN'].sel(z=chen_dryinflux['z'].where(chen_dryinflux['lag_QC']>1e-2).min()+30).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime just above cloud base in Chen_WEAKINV: {chen_weakinv['lag_ITMN'].sel(z=chen_weakinv['z'].where(chen_weakinv['lag_QC']>1e-2).min()+30).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime just above cloud base in Chen_DRY_WEAKINV: {chen_dryweakinv['lag_ITMN'].sel(z=chen_dryweakinv['z'].where(chen_dryweakinv['lag_QC']>1e-2).min()+30).sel(time = slice(1/12, 1/3)).mean():.3f} seconds")

In [ ]:
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_CONTROL: {chen_control['lag_ITMN'].sel(z=sub_startheight_chen+50).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_DRY: {chen_dry['lag_ITMN'].sel(z=sub_startheight_chendry+50).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_INFLUX: {chen_influx['lag_ITMN'].sel(z=sub_startheight_cheninflux+50).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_DRY_INFLUX: {chen_dryinflux['lag_ITMN'].sel(z=sub_startheight_chendryinflux+50).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_WEAKINV: {chen_weakinv['lag_ITMN'].sel(z=sub_startheight_weakinv+50).sel(time = slice(1/6, 1/3)).mean():.3f} seconds")
print(f"Ice mean lifetime 50 meters above sublimation start altitude in Chen_DRY_WEAKINV: {chen_dryweakinv['lag_ITMN'].sel(z=sub_startheight_dryweakinv+50).sel(time = slice(1/12, 1/3)).mean():.3f} seconds")

In [ ]:
print(f"Relative lifetime difference bvetween CONTROL and WEAKINV at sublimation height: {((chen_weakinv['lag_ITMN'].sel(z=sub_startheight_weakinv+50).sel(time = slice(1/6, 1/3)).mean()-chen_control['lag_ITMN'].sel(z=sub_startheight_chen+50).sel(time = slice(1/6, 1/3)).mean())/chen_control['lag_ITMN'].sel(z=sub_startheight_chen+50).sel(time = slice(1/6, 1/3)).mean()):.3%}")
print(f"Relative lifetime difference bvetween CONTROL and WEAKINV within cloud: {((chen_weakinv['lag_ITMN'].where(chen_weakinv['lag_QC']>1e-2).sel(time = slice(1/6, 1/3)).mean()-chen_control['lag_ITMN'].where(chen_control['lag_QC']>1e-2).sel(time = slice(1/6, 1/3)).mean())/chen_control['lag_ITMN'].where(chen_control['lag_QC']>1e-2).sel(time = slice(1/6, 1/3)).mean()):.3%}")


### Entrainment and Cloud Thickness

In [ ]:
cloudbase_influx = find_top(chen_influx["lag_QC"][::-1], 1e-2);
cloudbase_dryinflux = find_top(chen_dryinflux["lag_QC"][::-1], 1e-2)
cloudthick_influx = (cloudtop_influx - cloudbase_influx).sel(time = slice(1/12, 1/3)).mean()
cloudthick_dryinflux = (cloudtop_dryinflux - cloudbase_dryinflux).sel(time = slice(1/12, 1/3)).mean()
print(f"Average cloud thickness in Chen_CONTROL between hours 2 and 8: {cloudthick_control:.2f} m")
print(f"Average cloud thickness in Chen_DRY between hours 2 and 8: {cloudthick_dry:.2f} m")
print(f"Average cloud thickness in Chen_WEAKINV between hours 2 and 8: {cloudthick_weakinv:.2f} m")
print(f"Average cloud thickness in Chen_DRY_WEAKINV between hours 2 and 8: {cloudthick_dryweakinv:.2f} m")
print(f"Average cloud thickness in Chen_INFLUX between hours 2 and 8: {cloudthick_influx:.2f} m")
print(f"Average cloud thickness in Chen_DRY_INFLUX between hours 2 and 8: {cloudthick_dryinflux:.2f} m")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,3), dpi = 250)
axlabels = ["(a)", "(b)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i,ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_xlim(0,8)
    ax.axvline(1, color = "black", linestyle = ":")
ax1.set_ylim(0, 15)
ax2.set_ylim(100, 350)
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_control, 30)-subv, 20, mode = "nearest"), color = "blue", label = "CONTROL")
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_dry, 30)-subv, 20, mode = "nearest"), color = "darkorange", label = "DRY")
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_influx, 30)-subv, 20, mode = "nearest"), color = "blue", linestyle = "--", label = "INFLUX")
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_dryinflux, 30)-subv, 20, mode = "nearest"), color = "darkorange", linestyle = "--", label = "DRY_INFLUX")
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_weakinv, 30)-subv, 20, mode = "nearest"), color = "turquoise", label = "WEAKINV")
ax1.plot(24*chen_control["time"], 1000*gaussian_filter(np.gradient(cloudtop_dryweakinv, 30)-subv, 20, mode = "nearest"), color = "darkcyan", label = "DRY_WEAKINV")

ax2.plot(24*chen_control["time"], cloudtop_control - cloudbase_control, color = "blue")
ax2.plot(24*chen_control["time"], cloudtop_dry - cloudbase_dry, color = "darkorange")
ax2.plot(24*chen_control["time"], cloudtop_influx - cloudbase_influx, color = "blue", linestyle = "--")
ax2.plot(24*chen_control["time"], cloudtop_dryinflux - cloudbase_dryinflux, color = "darkorange", linestyle = "--")
ax2.plot(24*chen_control["time"], cloudtop_weakinv - cloudbase_weakinv, color = "turquoise")
ax2.plot(24*chen_control["time"], cloudtop_dryweakinv - cloudbase_dryweakinv, color = "darkcyan")

ax1.set_ylabel("Cloud Top Entrainment Rate ($mm \ s ^{-1}$)")
ax2.set_ylabel("Cloud Depth (m)")
ax2.set_xlabel("Elapsed Time (h)")
ax1.set_xlabel("Elapsed Time (h)")
fig.legend(ncols = 3, loc  = "outside lower center")
fig.savefig("paperfigs/supplemental_figure4_entrainment_cloudthick.png")
plt.close()

### Updraft-Differentiated Properties

In [ ]:
chen_dry.close(); del chen_dry
chen_influx.close(); del chen_influx
chen_dryinflux.close(); del chen_dryinflux
chen_weakinv.close(); del chen_weakinv
chen_dryweakinv.close(); del chen_dryweakinv

In [ ]:
out3dfolder = input("Enter the path to the folder containing the 3D model output converted to NetCDF format: ").strip()
chen_control_multitime = xr.open_dataset(f"{out3dfolder}/ISDAC_chen_control_merged.nc")

In [ ]:
control_updraft_qc = chen_control_multitime["QC"].where((chen_control_multitime["W"]>=0.5)*(chen_control_multitime["QC"]>1e-2))
control_downdraft_qc = chen_control_multitime["QC"].where((chen_control_multitime["W"]<=-0.5)*(chen_control_multitime["QC"]>1e-2))
control_updraft_qi = chen_control_multitime["QI"].where((chen_control_multitime["W"]>=0.5)*(chen_control_multitime["QC"]>1e-2))
control_downdraft_qi = chen_control_multitime["QI"].where((chen_control_multitime["W"]<=-0.5)*(chen_control_multitime["QC"]>1e-2))
control_mean_icephasefrac = chen_control_multitime["QI"].where(chen_control_multitime["QC"]>1e-2)/(chen_control_multitime["QI"].where(chen_control_multitime["QC"]>1e-2) + chen_control_multitime["QC"].where(chen_control_multitime["QC"]>1e-2))

In [ ]:
print(f"Average cloud liquid water content in updrafts at z=700m: {control_updraft_qc.sel(z = 700, method = 'nearest').mean().values:.3f} g/kg")
print(f"Average cloud liquid water content in downdrafts at z=700m: {control_downdraft_qc.sel(z = 700, method = 'nearest').mean().values:.3f} g/kg")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (9,3), dpi = 250, sharey = True)
axlabels = ["(a)", "(b)", "(c)"]
l,w,b,h = 0, 0.06, 0.94, 0.06
for i, ax in enumerate((ax1, ax2, ax3)):
    ax = add_subplotlabel(ax, axlabels[i], l,w,b,h)
    ax.set_ylim(0,1000)
ax1.set_ylabel("Altitude AMSL (m)")
ax2.set_xlabel("X-Distance (m)")
ax2.tick_params(left = False, labelleft = False, bottom = True, labelbottom = True)
ax2.set_xlabel("Ice Water Mix. Rat. ($g \ kg^{-1}$)")
ax3.tick_params(left = False, labelleft = False, bottom = True, labelbottom = True)
ax3.set_xlabel("Liquid Water Mix. Rat. ($g \ kg^{-1}$)")
from palettable.scientific.sequential import Devon_10
icemp = ax1.pcolormesh(chen_control_multitime["x"], chen_control_multitime["z"].sel(z = slice(0,1000)), chen_control_multitime["QI"].sel(z = slice(0,1000)).sel(time = 16260/86400, method = "nearest").mean(dim = "y"), vmin = 0, vmax = 2e-2, cmap = Devon_10.get_mpl_colormap().reversed())
ax1.contour(chen_control_multitime["x"], chen_control_multitime["z"].sel(z = slice(0,1000)), chen_control_multitime["W"].sel(z = slice(0,1000)).sel(time = 16200/86400, method = "nearest").mean(dim = "y"), vmin = -0.2, vmax = 0.2, levels = [-0.2, -0.1, -0.05, 0.05, 0.1, 0.2], cmap = "bwr", linestyles = "--")
ax1.contour(chen_control_multitime["x"], chen_control_multitime["z"].sel(z = slice(0,1000)), chen_control_multitime["QC"].sel(z = slice(0,1000)).sel(time = 16200/86400, method = "nearest").mean(dim = "y"), levels = [1e-2], colors = "black")
ax2.plot(control_updraft_qi.mean(dim = ("x", "y", "time")), chen_control_multitime["z"], color = "blue", label = "Updraft QI")
ax2.plot(control_downdraft_qi.mean(dim = ("x", "y", "time")), chen_control_multitime["z"], color = "blue", linestyle = "--", label = "Downdraft QI")
ax2.legend(loc = "lower left")
ax3.plot(control_updraft_qc.mean(dim = ("x", "y", "time")), chen_control_multitime["z"], color = "blue", label = "Updraft QC")
ax3.plot(control_downdraft_qc.mean(dim = ("x", "y", "time")), chen_control_multitime["z"], color = "blue", linestyle = "--", label = "Downdraft QC")
ax3.legend(loc = "lower left")
fig.colorbar(icemp, ax = [ax1, ax2, ax3], orientation = "horizontal", aspect = 35, fraction = 0.05, extend = "max", label = "Ice Water Mixing Ratio ($g \ kg^{-1}$)")
fig.savefig("paperfigs/updraftdowndraft_xc_timemean.png")
plt.close()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6, 3), dpi = 250)
axlabels = ["(a)", "(b)"]
for i, ax in enumerate((ax1, ax2)):
    ax = add_subplotlabel(ax, axlabels[i], 0, 0.06, 0.94, 0.06)
ax1.set_xlabel("Elapsed Time (h)")
ax1.set_ylabel("Ice Nucleation Rate ($mg \ kg^{-1} \ s^{-1}$")
ax1.set_yscale("log")
ax1.set_ylim(10**(-8), 10**(-5))
ax1.axvline(1, color = "black", linestyle = "--")
ax1.plot(24*chen_control["time"], chen_control["lag_NUC"].sel(z = slice(400,700)).where(chen_control["lag_NUC"].sel(z = slice(400,700))>1e-7).mean(dim = "z"), color = "blue", label = "Cloud Base Nucleation Rate")

ax1.plot(24*chen_control["time"], chen_control["lag_NUC"].sel(z = slice(700,900)).where(chen_control["lag_NUC"].sel(z = slice(700,900))>1e-9).mean(dim = "z"), color = "navy", label = "Cloud Top Nucleation Rate")
ax1.legend()
ax2.set_xlabel("Ice Phase Fraction")
ax2.set_ylabel("Altitude AMSL (m)")
ax2.set_ylim(550, 850)
ax2.set_xlim(0,0.4)
ax2.plot(control_mean_icephasefrac.mean(dim = ("x", "y", "time")), chen_control_multitime["z"], color = "black", label = "In-Cloud Average")
ax2.plot((control_updraft_qi/(control_updraft_qi + control_updraft_qc)).mean(dim = ("time", "x","y")), chen_control_multitime["z"], color = "red", linestyle = "--", label = "W > 0.5 $m \ s^{-1}$")
ax2.plot((control_downdraft_qi/(control_downdraft_qi + control_downdraft_qc)).mean(dim = ("time", "x","y")), chen_control_multitime["z"], color = "blue", linestyle = "--", label = "W < -0.5 $m \ s^{-1}$")
ax2.legend()
ax1.set_title("Time Series of Ice Nucleation Rate")
ax2.set_title("Vertical Profile of Ice Phase Fraction \n Averaged Between Hours 2 and 8")

fig.savefig(f"paperfigs/control_nucleation_phasefraction.png")
plt.close()

In [ ]:
chen_control_multitime.close(); del chen_control_multitime
control_updraft_qc.close(); del control_updraft_qc
control_downdraft_qc.close(); del control_downdraft_qc
control_updraft_qi.close(); del control_updraft_qi
control_downdraft_qi.close(); del control_downdraft_qi
control_mean_icephasefrac.close(); del control_mean_icephasefrac